In [3]:
# imports
import pandas as pd
import numpy as np
import numpy_financial as npf
import random  
import matplotlib.pyplot as plt
import time

from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO, DQN, A2C
from stable_baselines3.common.callbacks import CallbackList, CheckpointCallback, EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from sb3_contrib import RecurrentPPO

from environment_fx import calculate_import_export, test1, test2, test3, evaluate1, evaluate2, basepolicy

import gymnasium as gym
from gymnasium import spaces

import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from typing import Callable

import torch
import torch as th
from torch import nn
import torch.nn as nn

import rainflow

In [4]:
# import and modify data

# Assuming the file is a CSV and specifying the correct path and filename
file_path = r"C:\Users\kubaw\Desktop\DELFT\THESIS\CH5"

# Use pandas to read the CSV file
AC_OUTPUT = pd.read_csv(file_path + "/AC_OUTPUT_JA")
elec_df = pd.read_csv(file_path + "/hourly_consumption_gemany.csv")
import_price = pd.read_csv(file_path + "/electricity_tariff.csv")

#elec_df = elec_df * 1000
elec_df = elec_df.drop('HourOfYear', axis=1)

elec_df['hour_of_day'] = np.arange(8760) % 24
elec_df['day_of_week'] = np.arange(8760) // 24 % 7  # 0 is Monday, 6 is Sunday

# Define rates
peak_rate = 1.45
normal_rate = 1
off_peak_rate = 0.85

# Function to determine rate based on hour and day
def determine_rate(hour, day):
    if day < 5:  # Monday to Friday
        if 16 <= hour < 21:  # 4pm to 9pm
            return peak_rate
        elif 6 <= hour < 10:  # 7am to 9am and 10am to 3pm
            return normal_rate
        else:  # Off-peak times
            return off_peak_rate
    else:  # Weekend
        if 16 <= hour < 21:  # 4pm to 9pm
            return normal_rate
        else:  # Off-peak times
            return off_peak_rate
    
# Apply the function to each row to determine the rate
elec_df['rate'] = elec_df.apply(lambda row: determine_rate(row['hour_of_day'], row['day_of_week']), axis=1)

import_price_df = import_price.drop(columns=['x'])
import_price_df = import_price_df[:-26]

train_cols = random.sample(list(import_price_df.columns), 7000)
import_price_train = import_price_df[train_cols]
test_cols = [col for col in import_price_df.columns if col not in train_cols]
import_price_test = import_price_df[test_cols]

Eff = pd.read_csv(file_path + "/Efficency_impr")
Eff = (Eff)/100 + 1

CAPEX = pd.read_csv(file_path + "/CAPEX_JA.csv")
CAPEX_JA = (CAPEX[:26])

train_cols_CAPEX = random.sample(list(CAPEX_JA.columns), 7000)
test_cols_CAPEX = [col for col in CAPEX_JA.columns if col not in train_cols_CAPEX]

CAPEX_JA_train = CAPEX_JA[train_cols_CAPEX]
CAPEX_JA_test = CAPEX_JA[test_cols_CAPEX]

train_cols_Eff = random.sample(list(Eff.columns), 7000)
test_cols_Eff = [col for col in Eff.columns if col not in train_cols_Eff]

Eff_train = Eff[train_cols_Eff]
Eff_test = Eff[test_cols_Eff]

AC_OUTPUT_arr = (np.array(AC_OUTPUT.T)).flatten()

Eff_train_arr = np.array(Eff_train.T)
Eff_test_arr = np.array(Eff_test.T)

CAPEX_JA_train_arr = np.array(CAPEX_JA_train.T)
CAPEX_JA_test_arr = np.array(CAPEX_JA_test.T)

elec_consum_arr = np.array(elec_df["Consumption"])
import_price_rate = np.array(elec_df["rate"])

import_price_train_arr = np.array(import_price_train.T)
import_price_test_arr = np.array(import_price_train.T)

file_path2 = r"C:\Users\kubaw\Desktop\DELFT\THESIS\CH6"

Battery_CAPEX = pd.read_csv(file_path2 + "/batery_CAPES.csv")
Battery_CAPEX = (Battery_CAPEX[:26])


train_cols_Battery_CAPEX = random.sample(list(Battery_CAPEX.columns), 7000)
test_cols_Battery_CAPEX = [col for col in Battery_CAPEX.columns if col not in train_cols_Battery_CAPEX]

Battery_CAPEX_train = Battery_CAPEX[train_cols_Battery_CAPEX]
Battery_CAPEX_test = Battery_CAPEX[test_cols_Battery_CAPEX]

Battery_CAPEX_train_arr = np.array(Battery_CAPEX_train.T)
Battery_CAPEX_test_arr = np.array(Battery_CAPEX_test.T)

In [57]:
file_path3 = r"C:\Users\kubaw\Desktop\DELFT\THESIS\CH6"
battery_degradation_arr = pd.read_csv(file_path3 + "/batery_degradation_arr.csv")
battery_degradation_arr = battery_degradation_arr.T
split_point = int(0.3 * len(battery_degradation_arr))
battery_degradation_train = battery_degradation_arr.iloc[:split_point]
battery_degradation_train_arr = battery_degradation_train.to_numpy()

battery_degradation_test = battery_degradation_arr.iloc[split_point:]
battery_degradation_test_arr = battery_degradation_test.to_numpy()

In [61]:
battery_degradation_test_arr[10][365*10]

18.38345314558381

In [70]:
class TrainEnvironment(gym.Env):
    def __init__(self, AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_tariff, efficency, CAPEX, Battery_CAPEX, battery_degradation):
        
        # Price per watthour
        self.import_price_df = import_tariff
        self.import_price_at_zero = np.float32(0.00035)
        self.import_price_rate = import_price_rate
        
        # Energy Balance
        self.AC_OUTPUT = AC_OUTPUT_arr
        self.elec_df = elec_consum_arr
        self.max_export = 4000
        self.number_of_panels = 24
        
        # Degradation
        self.deg_mu = 1.055
        self.deg_std = 0.555
        
        # Efficency Development
        self.efficency_develop_df = efficency
        self.efficency_at_zero = 1.0
        
        # Costs
        self.power_at_zero = 415
        self.cost_per_Wp_df_at_zero = 0.69
        self.cost_per_Wp_df = CAPEX
        self.initial_other_costs = 150
        
        self.operational_cost = 16.8
        #self.budget_constraint = 1500
        
        self.loan_interest_rate = 1.06
        self.normal_interest_rate = 1.02
        
        self.battery_CAPEX = Battery_CAPEX
        self.battery_CAPEX_at_zero = 1
        
        self.possible_batery_sizes = 1
        
        #self.no_of_cycles = 365
        self.battery_degradation = battery_degradation
                        
        # Spaces and length
        self.action_space = spaces.MultiDiscrete([self.number_of_panels + 1, 2])
        
        self.observation_space = spaces.Box(0, 1.25, shape=(self.number_of_panels + 7 + 2,))
        self.episode_len = 25
        self.months_per_timestep = 12
        
    def _get_obs(self):
        
        return self.observation
    
    def count_cycles(self, SOC_array):
        
        cycles = list(rainflow.count_cycles(SOC_array))

        num_cycles = len(cycles)

        return cycles

    def calculate_import_export(self, AC_OUTPUT, elec_df, export_price, import_price):
        
        self.dis_max_charge = 1500
        self.dis_charge_eff = 0.95
        self.leak = self.batery_cap * 0.0001
        self.batery_cap = 5000
        self.current_battery_cap = self.batery_cap * self.battery_state
        self.leak = self.batery_cap * 0.0001
        self.min_SOC = 0.05 * self.batery_cap
        self.max_SOC = self.batery_cap 
        self.year_len = 8760
        SOC_array = []
        
        if self.battery_state == 0:
        
            AC_OUTPUT_tot = self._get_obs()[0:self.number_of_panels].sum() * self.AC_OUTPUT 

            exported = (AC_OUTPUT_tot - self.elec_df).clip(min=0, max = self.max_export)        
            export_revenue = (export_price * exported).sum()


            minimised = AC_OUTPUT_tot - exported 
            minimised_revenue = (minimised * (self.import_price_rate * import_price)).sum()
            
        else: 
            
            max_discharge = self.dis_max_charge * self.dis_charge_eff
            max_charge = self.dis_max_charge * self.dis_charge_eff
            
            
            for timestep in range(self.year_len):
                
                self.current_SOC -= self.leak 
                
                AC_OUTPUT_tot = self._get_obs()[0:self.number_of_panels].sum() * self.AC_OUTPUT
                
                pv = AC_OUTPUT_tot

                load = self.elec_df[timestep]
                elec_price = self.import_price_rate[timestep] * import_price

                if pv + self.current_SOC >= load:
                    direct_consumption = load
                    excess_pv = pv - direct_consumption
                    if excess_pv > 0:
                        potential_charge = min(excess_pv, max_charge)
                        if self.current_SOC + potential_charge > self.current_battery_cap:
                            to_grid = self.current_SOC + potential_charge - self.current_battery_cap
                            self.current_SOC = self.current_battery_cap
                        else:
                            self.current_SOC += potential_charge
                            to_grid = 0
                    else:
                        to_grid = 0
                    from_grid = 0
                else:
                    # Not enough energy from PV and battery, need grid power
                    from_pv_to_load = pv
                    needed_from_battery = load - from_pv_to_load
                    actual_discharge = min(needed_from_battery, max_discharge, self.current_SOC)
                    direct_consumption = from_pv_to_load + actual_discharge
                    self.current_SOC -= actual_discharge
                    if load > direct_consumption:
                        from_grid = load - direct_consumption
                    else:
                        from_grid = 0
                        
                SOC_array.append(current_SOC)

                # Savings from direct consumption
                minimised_revenue = direct_consumption * elec_price
                export_revenue += to_grid * export_price
            


        return export_revenue, AC_OUTPUT_tot, minimised_revenue, SOC_array    
    
    def reset(self, seed=None):
        
        """
        Reset the environment to the original state at t=1
        """
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)
        
        # Panels
        self.init_obs = np.random.uniform(0, 1, size=self.number_of_panels).astype(np.float32)
        self.init_obs = np.where(self.init_obs < 0.5, 0.0, np.random.uniform(0.85, 1.0, size=self.number_of_panels))

        # Combine all initialization into a single step for efficiency
        self.import_price_at_zero_norm = (self.import_price_at_zero - 0.00022499) / (0.0020798 - 0.00022499)
        self.FiT_at_zero_norm = (self.import_price_at_zero - 0.00022499 * 0.33) / (0.0020798 - 0.00022499 * 0.33)
        self.efficency_at_zero_norm = (self.efficency_at_zero - 0.999) / (1.156 - 0.999)
        self.panel_cost_and_inverter_at_zero_norm = (self.cost_per_Wp_df_at_zero - 0.1712159) / (0.8214356 - 0.1712159)
        
        self.current_budget_constraint = np.random.randint(750, 2000)
        self.next_step_budget_constraint = 0
        
        self.battery_CAPEX_at_zero_norm = (self.battery_CAPEX_at_zero - 0.03) / (4 - 0.03)
        
        # Complete observation initialization in one go
        self.observation = np.concatenate([
            self.init_obs,
            [self.import_price_at_zero_norm, self.FiT_at_zero_norm, self.efficency_at_zero_norm, 
             self.panel_cost_and_inverter_at_zero_norm, 0., 0., 0., 0., self.battery_CAPEX_at_zero_norm]
        ]).astype(np.float32)

        self.previous_observation = self.observation.copy()

        # RANDOM IMPORT PRICE
        self.random_import_price = self.import_price_df[np.random.choice(self.import_price_df.shape[0])] 

        # RANDOM EFFICENCY
        self.random_efficency_develop = self.efficency_develop_df[np.random.choice(self.efficency_develop_df.shape[0])]   
        
        # RANDOM COST PER WP
        self.random_cost_per_Wp = self.cost_per_Wp_df[np.random.choice(self.cost_per_Wp_df.shape[0])]   
        
        self.random_battery_CAPEX = self.battery_CAPEX[np.random.choice(self.battery_CAPEX.shape[0])]
        
        self.episode_len = 25  
    
        info = {}
        
        self.current_SOC = 0
        
        # RESET BALANCES
        self.fin_balance_tot = 0
        self.reward_tot = 0
        self.env_balance_tot = 0
        self.produced = 0
        self.other_costs = 0
        self.FiT = 0.0004
        self.next_FiT = 0.0004

        self.total_cash_flow = []
        self.annual_cash_flow = 0
                
        self.due_loans = [0, 0, 0, 0] 
        self.current_interest = 0
        self.step_total_interest = 1
        self.survival = np.zeros(self.number_of_panels, dtype=np.float32)
        self.resale_values = array_of_zeros = np.zeros(self.number_of_panels, dtype=np.float32)
        
        self.broke = np.zeros(self.number_of_panels, dtype=np.float32)
        
        self.two_year_ago_interest = 0
        self.first_year_interest = []
        self.second_year_interest = [0]
        self.third_year_interest = [0, 0]
        self.fourth_year_interest = [0, 0, 0]
        self.next_year_total = 0
        
        self.battery_state = 0
        
        self.survival = np.zeros(self.number_of_panels, dtype=np.float32)
    
        return self.observation, info
    
    def calculate_resale(self, initial_panel_cost, indices):
        
        self.resale_values[indices] = initial_panel_cost
        
        self.resale_values = self.resale_values * 0.85
        
        for count, i in enumerate(self.broke):
            if i == 1:
                self.resale_values[count] = 0
        
        resale_step = self.resale_values[indices].sum()
        
        return resale_step
    
    def calculate_panel_inv_cost(self, cost_per_Wp):
        
        PW_ep = self.efficency_develop * self.power_at_zero
        
        panel_cost_and_inverter = PW_ep * cost_per_Wp
        
        return panel_cost_and_inverter
        
    def calculate_penalty(self, current_step, annual_expense):
              
        year = 25 - current_step
        
        if year > 0:
            self.current_budget_constraint = self.next_step_budget_constraint    
            
        
        self.current_interest = self.next_year_total
        annual_expense = (-annual_expense)
        value = 0 
        loan = 0
        annual_interest = 0

        if annual_expense > self.current_budget_constraint:
            loan = (self.current_budget_constraint - annual_expense)
            value = annual_expense / self.current_budget_constraint
            periods = 2 if value < 2 else 3 if value < 3 else 4

            annual_interest = loan / periods
            interest_multiplier = 1

            for i in range(4):
                if i < periods:
                    self.due_loans[i] = annual_interest * interest_multiplier
                    interest_multiplier *= self.loan_interest_rate
                else:
                    self.due_loans[i] = 0
        else:
             self.due_loans = [0, 0, 0, 0]
    
        self.first_year_interest.append(self.due_loans[0])
        self.second_year_interest.append(self.due_loans[1])
        self.third_year_interest.append(self.due_loans[2])
        self.fourth_year_interest.append(self.due_loans[3])
    
    
        self.next_year_total = self.first_year_interest[year] + self.second_year_interest[year] + self.third_year_interest[year] + self.fourth_year_interest[year]
        
        self.next_step_budget_constraint = np.random.randint(750, 2000) * self.step_total_interest
        current_budget_observation = (self.next_step_budget_constraint - 750 * self.step_total_interest) / (2000 * self.step_total_interest - 750 * self.step_total_interest) 
        self.observation[self.number_of_panels + 6] = current_budget_observation
                
        return self.current_interest, self.due_loans, self.next_year_total
    
    def determine_battery_degradation(self, action_battery, no_cycles):
        
        if action_battery == 1:
            battery_state = 1
            random_column_index = np.random.randint(self.battery_degradation.shape[1])
            degradation_path = self.battery_degradation[:, random_column_index]
            year_of_install = abs(self.episode_len - 25)
            
        if self.observation[31] > 0:
            year_since_install = abs(year_of_install - 25)
            current_degradation = degradation_path[year_since_install * no_cycles]
            battery_state -= year_since_install/100
        
        return battery_state

    def calculate_total_CAPEX(self, action_step, panel_cost_and_inverter):
        """
        Calculate CAPEX each step in a vectorized manner.
        """
        BOS = panel_cost_and_inverter * 0.55
        number_installed = int(np.sum(action_step))

        # Calculate costs from module and inverter
        panel_cost_and_inverter_step = panel_cost_and_inverter * number_installed

        # Calculate other installation costs
        if number_installed == 0:
            other_costs = 0
        elif number_installed == 1:
            other_costs = self.initial_other_costs * self.step_total_interest
        else:
            discounts = 0.9 ** np.arange(number_installed)
            other_costs = (self.initial_other_costs * self.step_total_interest * discounts).sum()

        # Calculate BOS costs using vector operations
        is_new_installation = (self.previous_observation[:number_installed] == 0) & (action_step[:number_installed] == 1)
        is_replacement = (self.previous_observation[:number_installed] > 0) & (action_step[:number_installed] == 1)
        BOS_cost = np.sum(BOS * is_new_installation) + np.sum((BOS / 2) * is_replacement)

        # Sum total CAPEX
        total_CAPEX = panel_cost_and_inverter_step + BOS_cost + other_costs

        return total_CAPEX, panel_cost_and_inverter
        
    def failure(self, actions):
        
        beta = 3  # Shape parameter
        phi = 30  # Scale parameter  

        # Determine which panels are active based on the actions and previous observations.
        if self.episode_len == 24:
            active_panels = (self.observation[:self.number_of_panels] > 0.85)
        else:
            active_panels = (self.observation[:self.number_of_panels] == self.efficency_develop)

        # Calculate lifespan for all active panels at once
        lifespans = np.random.weibull(beta, self.number_of_panels) * phi
        lifespans = np.where(active_panels, lifespans, 0)  # Apply lifespan only to active panels

        # Adjust survival times based on episode length
        self.survival[:self.number_of_panels] = np.where(
            active_panels,
            np.abs(lifespans.astype(int)) + np.abs(self.episode_len - 25),
            self.survival[:self.number_of_panels]
        )

        return self.survival

    def calculate_FiT(self, episodes, import_price):
            
        self.FiT = import_price
            
        if episodes == 25:
            self.FiT = self.FiT
            
        elif episodes == 24 or episodes == 23:
            self.FiT = self.FiT * 0.64
            
        elif episodes == 22:
            self.FiT = self.FiT * 0.46
            
        elif episodes == 21:
            self.FiT = self.FiT * 0.55
            
        elif episodes < 20:
            self.FiT = self.FiT * 0.33
            
        elif episodes == 20:
            self.FiT = self.FiT * 0.37
            
        return self.FiT
                        
    def calculate_battery_CAPEX(self, action_battery):
        
        five_kw_PRICE = 3000

        installation = 1000 * self.step_total_interest
        
        this_battery_CAPEX = five_kw_PRICE * self.step_battery_CAPEX
        
        battery_bos = this_battery_CAPEX * 0.5
        
        if self.previous_observation[19] == 0:
            battery_bos = this_battery_CAPEX * 0.5
        else:
            battery_bos = 0
        
        total_batery_cost = (this_battery_CAPEX +  battery_bos + installation) * action_battery
        
        return total_batery_cost
        
    def step(self, action):
        
        """
        defines actions, reward etc.
        """
        
        # RESET THE ANNUAL BALANCES
        self.total_CAPEX = 0
        self.pv_costs = 0
        self.fin_balance = 0
        self.number_installed = 0
        current_penalty = 0
        self.other_costs = 0
        battery_costs = 0
        no_cycles = 0
        next_step_penalty = 0
        self.step_total_interest = self.step_total_interest * self.normal_interest_rate
        current_operational_costs = self.operational_cost * self.step_total_interest
        
        #***
        self.step_battery_CAPEX = self.random_battery_CAPEX[abs(self.episode_len - 25)]
        
        self.cost_per_Wp = self.random_cost_per_Wp[abs(self.episode_len - 25)]
        self.import_price = self.random_import_price[abs(self.episode_len - 25)]
        self.efficency_develop = self.random_efficency_develop[abs(self.episode_len - 25)]
           
        self.panel_cost_and_inverter = self.calculate_panel_inv_cost(self.cost_per_Wp)
        FiT = self.calculate_FiT(self.episode_len, self.import_price)
        
        reward = 0   
        actions_step = np.random.rand(12)
        
        #  ***
        action_battery = action[1]
        action = action[0]
        
        # Find indices of the lowest 'action' values in previous_observation
        indices = np.argsort(self.previous_observation[:self.number_of_panels])[:action]

        # Replace these indices in the observation with efficiency_develop
        self.observation[:self.number_of_panels][indices] = self.efficency_develop

        # Copy over the other values from previous_observation to observation
        mask = np.ones(len(self.previous_observation[:self.number_of_panels]), dtype=bool)
        mask[indices] = False
        self.observation[:self.number_of_panels][mask] = self.previous_observation[:self.number_of_panels][mask]

        replaced_panels = np.zeros(len(self.previous_observation[:self.number_of_panels]), dtype=int)
        replaced_panels[indices] = 1

        instaltion = (self.observation[:self.number_of_panels] > 0).astype(int)
        self.pv_costs -= instaltion.sum() * current_operational_costs

        actions_step = np.array(replaced_panels)
        
        if action_battery == 1:           
            self.observation[31] = 1.
            self.battery_state = 1
            
        if action > 0:
            step_CAPEX, panel_cost_and_inverter = self.calculate_total_CAPEX(actions_step, self.panel_cost_and_inverter)
            self.pv_costs -= step_CAPEX
            
        else:
            panel_cost_and_inverter = 0
            
        battery_costs = self.calculate_battery_CAPEX(action_battery)
        
        self.pv_costs -= battery_costs

        
        next_observation = self._get_obs()

        # Calculate the Reslae value
        resale = self.calculate_resale(panel_cost_and_inverter, indices)
        
        self.pv_costs += resale
 
        
        # CALCULATE THE BUDGET INTEREST
        current_penalty, due_loans, next_step_penalty = self.calculate_penalty(self.episode_len, self.pv_costs)

        
        # CALCULATE THE ENERGY YIELD
        exported_revenue, AC_OUTPUT_tot, minimised_revenue, SOC_array = self.calculate_import_export(self.AC_OUTPUT, 
                                                                          self.elec_df, FiT, self.import_price)        
        
        no_cycles = self.count_cycles(SOC_array)
        pv_costs_observation = - self.pv_costs / 9000
        self.observation[self.number_of_panels + 4] = pv_costs_observation
        
        next_step_penalty_observation = - next_step_penalty / 7000
        self.observation[self.number_of_panels + 5] = next_step_penalty_observation
        
        
        # CALCULATE STEP BALANCES
        self.fin_balance += self.pv_costs
        self.fin_balance += current_penalty
        self.fin_balance += float(exported_revenue + minimised_revenue)
        
        # CALCULATE TOTAL BALANCES
        self.fin_balance_tot += self.fin_balance                
        
        # SUBSTRACT 1 FOR TIMESTEP
        self.episode_len -= 1
        done = self.episode_len <= 0
        
        #reward = self.fin_balance_tot / 1000 if done else 0
        reward = self.fin_balance / 1000
        
        # FAILURE
        self.broke = np.zeros(self.number_of_panels, dtype=np.float32)
        survival = self.failure(actions_step)
        
        for c, p in enumerate(survival):
            
            if c < self.number_of_panels:

                if p - 1 <= abs(self.episode_len - 24):
                    self.broke[c] = 1
                    self.observation[c] = 0
        
        # DEGRADATION RATE
        # Applying degradation only to panels that are operational (above 0.1 efficiency)
        active_panels = self.observation[:self.number_of_panels] > 0.1
        degradations = np.random.normal(self.deg_mu, self.deg_std, size=self.number_of_panels) / 100
        self.observation[:self.number_of_panels][active_panels] -= degradations[active_panels]
        
        # BATERY DEGRADATION
        self.battery_state = determine_battery_degradation(action_battery, no_cycles)
        
        self.observation[31] = self.battery_state
        
        if not done: 
        
            self.next_cost_per_Wp = self.random_cost_per_Wp[abs(self.episode_len - 25)]
            self.next_import_price = self.random_import_price[abs(self.episode_len - 25)]
            self.next_efficency_develop = self.random_efficency_develop[abs(self.episode_len - 25)]
            next_FIT = self.calculate_FiT(self.episode_len, self.next_import_price)
        
            price_observation = (self.next_import_price - 0.00022499) / (0.0020798 - 0.00022499)
            self.observation[self.number_of_panels] = price_observation

            FiT_observation = (next_FIT - 0.00022499 * 0.33) / (0.0020798 - 0.00022499 * 0.33)
            self.observation[self.number_of_panels + 1] = FiT_observation

            eff_observation = (self.next_efficency_develop - 0.999) / (1.156 - 0.999)
            self.observation[self.number_of_panels + 2] = eff_observation

            cost_per_Wp_observation = (self.next_cost_per_Wp - 0.1712159) / (0.8214356 - 0.1712159) 
            self.observation[self.number_of_panels + 3] = cost_per_Wp_observation
            
            step_battery_CAPEX_obs = (self.step_battery_CAPEX - 0.03) / (4 - 0.03)
            self.observation[32] = step_battery_CAPEX_obs
        
        info = {"step financial balance (eur):": battery_costs,
               "total financial balance: (eur)": minimised_revenue,
               "internal rate of return": 0,
               "current_interest": resale,
                "net present value": 0}
         
        
        self.previous_observation = self.observation.copy()
        
        return self.observation, reward, done, False, info

In [71]:
class TestEnvironment(gym.Env):
    def __init__(self, AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_tariff, efficency, CAPEX, Battery_CAPEX,battery_degradation):
        
        # Price per watthour
        self.import_price_df = import_tariff
        self.import_price_at_zero = np.float32(0.00035)
        self.import_price_rate = import_price_rate
        
        # Energy Balance
        self.AC_OUTPUT = AC_OUTPUT_arr
        self.elec_df = elec_consum_arr
        self.max_export = 4000
        self.number_of_panels = 24
        
        # Degradation
        self.deg_mu = 1.055
        self.deg_std = 0.555
        
        # Efficency Development
        self.efficency_develop_df = efficency
        self.efficency_at_zero = 1.0
        
        # Costs
        self.power_at_zero = 415
        self.cost_per_Wp_df_at_zero = 0.69
        self.cost_per_Wp_df = CAPEX
        self.initial_other_costs = 150
        
        self.operational_cost = 16.8
        #self.budget_constraint = 1500
        
        self.no_of_cycles = 365
        self.battery_degradation = battery_degradation
        self.loan_interest_rate = 1.06
        self.normal_interest_rate = 1.02
        
        self.battery_CAPEX = Battery_CAPEX
        self.battery_CAPEX_at_zero = 1
            
        # Spaces and length
        self.action_space = spaces.MultiDiscrete([self.number_of_panels + 1, 2])
        self.observation_space = spaces.Box(0, 1.25, shape=(self.number_of_panels + 7 + 2,))
        self.episode_len = 25
        self.months_per_timestep = 12
        
    def _get_obs(self):
        
        return self.observation
    
    def count_cycles(self, SOC_array):
        
        cycles = list(rainflow.count_cycles(SOC_array))

        num_cycles = len(cycles)

        return cycles

    def calculate_import_export(self, AC_OUTPUT, elec_df, export_price, import_price):
        
        self.dis_max_charge = 1500
        self.dis_charge_eff = 0.95
        self.leak = self.batery_cap * 0.0001
        self.batery_cap = 5000
        self.current_battery_cap = self.batery_cap * self.battery_state
        self.leak = self.batery_cap * 0.0001
        self.min_SOC = 0.05 * self.batery_cap
        self.max_SOC = self.batery_cap 
        self.year_len = 8760
        SOC_array = []
        
        if self.battery_state == 0:
        
            AC_OUTPUT_tot = self._get_obs()[0:self.number_of_panels].sum() * self.AC_OUTPUT 

            exported = (AC_OUTPUT_tot - self.elec_df).clip(min=0, max = self.max_export)        
            export_revenue = (export_price * exported).sum()


            minimised = AC_OUTPUT_tot - exported 
            minimised_revenue = (minimised * (self.import_price_rate * import_price)).sum()
            
        else: 
            
            max_discharge = self.dis_max_charge * self.dis_charge_eff
            max_charge = self.dis_max_charge * self.dis_charge_eff
            
            
            for timestep in range(self.year_len):
                
                self.current_SOC -= self.leak 
                
                AC_OUTPUT_tot = self._get_obs()[0:self.number_of_panels].sum() * self.AC_OUTPUT
                
                pv = AC_OUTPUT_tot

                load = self.elec_df[timestep]
                elec_price = self.import_price_rate[timestep] * import_price

                if pv + self.current_SOC >= load:
                    direct_consumption = load
                    excess_pv = pv - direct_consumption
                    if excess_pv > 0:
                        potential_charge = min(excess_pv, max_charge)
                        if self.current_SOC + potential_charge > self.current_battery_cap:
                            to_grid = self.current_SOC + potential_charge - self.current_battery_cap
                            self.current_SOC = self.current_battery_cap
                        else:
                            self.current_SOC += potential_charge
                            to_grid = 0
                    else:
                        to_grid = 0
                    from_grid = 0
                else:
                    # Not enough energy from PV and battery, need grid power
                    from_pv_to_load = pv
                    needed_from_battery = load - from_pv_to_load
                    actual_discharge = min(needed_from_battery, max_discharge, self.current_SOC)
                    direct_consumption = from_pv_to_load + actual_discharge
                    self.current_SOC -= actual_discharge
                    if load > direct_consumption:
                        from_grid = load - direct_consumption
                    else:
                        from_grid = 0
                        
                SOC_array.append(current_SOC)

                # Savings from direct consumption
                minimised_revenue = direct_consumption * elec_price
                export_revenue += to_grid * export_price
            


        return export_revenue, AC_OUTPUT_tot, minimised_revenue, SOC_array    
    
    def reset(self, seed=None):
        
        """
        Reset the environment to the original state at t=1
        """
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)
            
        # Panels
        self.init_obs = np.random.uniform(0, 1, size=self.number_of_panels).astype(np.float32)
        self.init_obs = np.where(self.init_obs < 0.5, 0.0, np.random.uniform(0.85, 1.0, size=self.number_of_panels))

        # Combine all initialization into a single step for efficiency
        self.import_price_at_zero_norm = (self.import_price_at_zero - 0.00022499) / (0.0020798 - 0.00022499)
        self.FiT_at_zero_norm = (self.import_price_at_zero - 0.00022499 * 0.33) / (0.0020798 - 0.00022499 * 0.33)
        self.efficency_at_zero_norm = (self.efficency_at_zero - 0.999) / (1.156 - 0.999)
        self.panel_cost_and_inverter_at_zero_norm = (self.cost_per_Wp_df_at_zero - 0.1712159) / (0.8214356 - 0.1712159)
        
        self.current_budget_constraint = np.random.randint(750, 2000)
        self.next_step_budget_constraint = 0
        
        
        self.battery_CAPEX_at_zero_norm = (self.battery_CAPEX_at_zero - 0.03) / (4 - 0.03)
        
        # Complete observation initialization in one go
        self.observation = np.concatenate([
            self.init_obs,
            [self.import_price_at_zero_norm, self.FiT_at_zero_norm, self.efficency_at_zero_norm, 
             self.panel_cost_and_inverter_at_zero_norm, 0., 0., 0., 0., self.battery_CAPEX_at_zero_norm]
        ]).astype(np.float32)

        self.previous_observation = self.observation.copy()

        # RANDOM IMPORT PRICE
        self.random_import_price = self.import_price_df[np.random.choice(self.import_price_df.shape[0])] 

        # RANDOM EFFICENCY
        self.random_efficency_develop = self.efficency_develop_df[np.random.choice(self.efficency_develop_df.shape[0])]   
        
        # RANDOM COST PER WP
        self.random_cost_per_Wp = self.cost_per_Wp_df[np.random.choice(self.cost_per_Wp_df.shape[0])]   
        
        self.random_battery_CAPEX = self.battery_CAPEX[np.random.choice(self.battery_CAPEX.shape[0])]
        
        self.episode_len = 25  
    
        info = {}
        
        self.current_SOC = 0
        
        # RESET BALANCES
        self.fin_balance_tot = 0
        self.reward_tot = 0
        self.env_balance_tot = 0
        self.produced = 0
        self.other_costs = 0
        self.FiT = 0.0004
        self.next_FiT = 0.0004
        self.resale_values = array_of_zeros = np.zeros(self.number_of_panels, dtype=np.float32)
        
        self.broke = np.zeros(self.number_of_panels, dtype=np.float32)
        self.total_cash_flow = []
        self.annual_cash_flow = 0
                
        self.due_loans = [0, 0, 0, 0] 
        self.current_interest = 0
        self.step_total_interest = 1
        
        self.two_year_ago_interest = 0
        self.first_year_interest = []
        self.second_year_interest = [0]
        self.third_year_interest = [0, 0]
        self.fourth_year_interest = [0, 0, 0]
        self.next_year_total = 0
        self.battery_state = 0

        self.survival = np.zeros(self.number_of_panels, dtype=np.float32)
    
        return self.observation, info
    
    def calculate_resale(self, initial_panel_cost, indices):
        
        self.resale_values[indices] = initial_panel_cost
        
        self.resale_values = self.resale_values * 0.85
        
        for count, i in enumerate(self.broke):
            if i == 1:
                self.resale_values[count] = 0
        
        resale_step = self.resale_values[indices].sum()
        
        return resale_step
    
    def calculate_panel_inv_cost(self, cost_per_Wp):
        
        PW_ep = self.efficency_develop * self.power_at_zero
        
        panel_cost_and_inverter = PW_ep * cost_per_Wp
        
        return panel_cost_and_inverter
    
    def calculate_irr_and_npv(self, pv_cost, minimised_revenue, export_revenue, penalty):
                
        """
        Calculates total cash flow of the project needed for the internal rate of return
        """ 
        self.expences = 0
        self.annual_cash_flow = 0
        initial_cost = 0
        
        self.expences = pv_cost
        self.annual_cash_flow = self.expences + export_revenue + minimised_revenue + penalty
        initial_cost_q, x = self.calculate_total_CAPEX(self.init_obs, self.panel_cost_and_inverter)
        initial_cost = - initial_cost_q
        
        if self.episode_len == 24:
            self.total_cash_flow.append(initial_cost + self.annual_cash_flow) 
        else:
            self.total_cash_flow.append(self.annual_cash_flow) 
        
        return self.total_cash_flow
        
    def calculate_penalty(self, current_step, annual_expense):
              
        year = 25 - current_step
        
        if year > 0:
            self.current_budget_constraint = self.next_step_budget_constraint    
            
        
        self.current_interest = self.next_year_total
        annual_expense = (-annual_expense)
        value = 0 
        loan = 0
        annual_interest = 0

        if annual_expense > self.current_budget_constraint:
            loan = (self.current_budget_constraint - annual_expense)
            value = annual_expense / self.current_budget_constraint
            periods = 2 if value < 2 else 3 if value < 3 else 4

            annual_interest = loan / periods
            interest_multiplier = 1

            for i in range(4):
                if i < periods:
                    self.due_loans[i] = annual_interest * interest_multiplier
                    interest_multiplier *= self.loan_interest_rate
                else:
                    self.due_loans[i] = 0
        else:
             self.due_loans = [0, 0, 0, 0]
    
        self.first_year_interest.append(self.due_loans[0])
        self.second_year_interest.append(self.due_loans[1])
        self.third_year_interest.append(self.due_loans[2])
        self.fourth_year_interest.append(self.due_loans[3])
    
    
        self.next_year_total = self.first_year_interest[year] + self.second_year_interest[year] + self.third_year_interest[year] + self.fourth_year_interest[year]
        
        self.next_step_budget_constraint = np.random.randint(750, 2000) * self.step_total_interest
        current_budget_observation = (self.next_step_budget_constraint - 750 * self.step_total_interest) / (2000 * self.step_total_interest - 750 * self.step_total_interest) 
        self.observation[self.number_of_panels + 6] = current_budget_observation
                
        return self.current_interest, self.due_loans, self.next_year_total
    
    def determine_battery_degradation(self, action_battery, no_cycles):
        
        if action_battery == 1:
            battery_state = 1
            random_column_index = np.random.randint(self.battery_degradation.shape[1])
            degradation_path = self.battery_degradation[:, random_column_index]
            year_of_install = abs(self.episode_len - 25)
            
        if self.observation[31] > 0:
            year_since_install = abs(year_of_install - 25)
            current_degradation = degradation_path[year_since_install * no_cycles]
            battery_state -= year_since_install/100
        
        return battery_state
        
    def calculate_total_CAPEX(self, action_step, panel_cost_and_inverter):
        """
        Calculate CAPEX each step in a vectorized manner.
        """
        BOS = panel_cost_and_inverter * 0.55
        number_installed = int(np.sum(action_step))

        # Calculate costs from module and inverter
        panel_cost_and_inverter_step = panel_cost_and_inverter * number_installed

        # Calculate other installation costs
        if number_installed == 0:
            other_costs = 0
        elif number_installed == 1:
            other_costs = self.initial_other_costs * self.step_total_interest
        else:
            discounts = 0.9 ** np.arange(number_installed)
            other_costs = (self.initial_other_costs * self.step_total_interest * discounts).sum()

        # Calculate BOS costs using vector operations
        is_new_installation = (self.previous_observation[:number_installed] == 0) & (action_step[:number_installed] == 1)
        is_replacement = (self.previous_observation[:number_installed] > 0) & (action_step[:number_installed] == 1)
        BOS_cost = np.sum(BOS * is_new_installation) + np.sum((BOS / 2) * is_replacement)

        # Sum total CAPEX
        total_CAPEX = panel_cost_and_inverter_step + BOS_cost + other_costs

        return total_CAPEX, panel_cost_and_inverter
        
    def failure(self, actions):
        
        beta = 3  # Shape parameter
        phi = 30  # Scale parameter  

        # Determine which panels are active based on the actions and previous observations.
        if self.episode_len == 24:
            active_panels = (self.observation[:self.number_of_panels] > 0.85)
        else:
            active_panels = (self.observation[:self.number_of_panels] == self.efficency_develop)

        # Calculate lifespan for all active panels at once
        lifespans = np.random.weibull(beta, self.number_of_panels) * phi
        lifespans = np.where(active_panels, lifespans, 0)  # Apply lifespan only to active panels

        # Adjust survival times based on episode length
        self.survival[:self.number_of_panels] = np.where(
            active_panels,
            np.abs(lifespans.astype(int)) + np.abs(self.episode_len - 25),
            self.survival[:self.number_of_panels]
        )

        return self.survival

    def calculate_FiT(self, episodes, import_price):
            
        self.FiT = import_price
            
        if episodes == 25:
            self.FiT = self.FiT
            
        elif episodes == 24 or episodes == 23:
            self.FiT = self.FiT * 0.64
            
        elif episodes == 22:
            self.FiT = self.FiT * 0.46
            
        elif episodes == 21:
            self.FiT = self.FiT * 0.55
            
        elif episodes < 20:
            self.FiT = self.FiT * 0.33
            
        elif episodes == 20:
            self.FiT = self.FiT * 0.37
            
        return self.FiT
    
    def calculate_battery_CAPEX(self, action_battery):
        
        five_kw_PRICE = 3000

        installation = 1000 * self.step_total_interest
        
        this_battery_CAPEX = five_kw_PRICE * self.step_battery_CAPEX
        
        battery_bos = this_battery_CAPEX * 0.5
        
        if self.previous_observation[19] == 0:
            battery_bos = this_battery_CAPEX * 0.5
        else:
            battery_bos = 0
        
        total_batery_cost = (this_battery_CAPEX +  battery_bos + installation) * action_battery
        
        return total_batery_cost
                  
    def step(self, action):
        
        """
        defines actions, reward etc.
        """
        
        # RESET THE ANNUAL BALANCES
        self.total_CAPEX = 0
        self.pv_costs = 0
        self.fin_balance = 0
        self.number_installed = 0
        irr_fin = 0
        npv_fin = 0
        battery_costs = 0
        current_penalty = 0
        self.other_costs = 0
        next_step_penalty = 0
        self.step_total_interest = self.step_total_interest * self.normal_interest_rate
        current_operational_costs = self.operational_cost * self.step_total_interest
        self.step_battery_CAPEX = self.random_battery_CAPEX[abs(self.episode_len - 25)]

        
        self.cost_per_Wp = self.random_cost_per_Wp[abs(self.episode_len - 25)]
        self.import_price = self.random_import_price[abs(self.episode_len - 25)]
        self.efficency_develop = self.random_efficency_develop[abs(self.episode_len - 25)]
           
        self.panel_cost_and_inverter = self.calculate_panel_inv_cost(self.cost_per_Wp)
        FiT = self.calculate_FiT(self.episode_len, self.import_price)
        
        reward = 0   
        actions_step = np.random.rand(self.number_of_panels + 1)
        
        
        action_battery = action[1]
        action = action[0]
        
        # Find indices of the lowest 'action' values in previous_observation
        indices = np.argsort(self.previous_observation[:self.number_of_panels])[:action]

        # Replace these indices in the observation with efficiency_develop
        self.observation[:self.number_of_panels][indices] = self.efficency_develop

        # Copy over the other values from previous_observation to observation
        mask = np.ones(len(self.previous_observation[:self.number_of_panels]), dtype=bool)
        mask[indices] = False
        self.observation[:self.number_of_panels][mask] = self.previous_observation[:self.number_of_panels][mask]

        replaced_panels = np.zeros(len(self.previous_observation[:self.number_of_panels]), dtype=int)
        replaced_panels[indices] = 1

        instaltion = (self.observation[:self.number_of_panels] > 0).astype(int)
        self.pv_costs -= instaltion.sum() * current_operational_costs

        actions_step = np.array(replaced_panels)
        
        if action_battery == 1:           
            self.observation[31] = 1.
            self.battery_state = 1
            
        if action > 0:
            step_CAPEX, panel_cost_and_inverter = self.calculate_total_CAPEX(actions_step, self.panel_cost_and_inverter)
            self.pv_costs -= step_CAPEX
            
        else:
            panel_cost_and_inverter = 0
            
        battery_costs = self.calculate_battery_CAPEX(action_battery)
        
        self.pv_costs -= battery_costs
        next_observation = self._get_obs()

        # Calculate the Reslae value
        resale = self.calculate_resale(panel_cost_and_inverter, indices) #  ***
        
        self.pv_costs += resale

        
        # CALCULATE THE BUDGET INTEREST
        current_penalty, due_loans, next_step_penalty = self.calculate_penalty(self.episode_len, self.pv_costs)
        
        
        # CALCULATE THE ENERGY YIELD
        exported_revenue, AC_OUTPUT_tot, minimised_revenue, SOC_array = self.calculate_import_export(self.AC_OUTPUT, 
                                                                          self.elec_df, FiT, self.import_price)        
        
        no_cycles = self.count_cycles(SOC_array)
        
        pv_costs_observation = - self.pv_costs / 9000
        self.observation[self.number_of_panels + 4] = pv_costs_observation
        
        next_step_penalty_observation = - next_step_penalty / 7000
        self.observation[self.number_of_panels + 5] = next_step_penalty_observation
        
        
        # CALCULATE STEP BALANCES
        self.fin_balance += self.pv_costs
        self.fin_balance += current_penalty
        self.fin_balance += float(exported_revenue + minimised_revenue)
        
        # CALCULATE TOTAL BALANCES
        self.fin_balance_tot += self.fin_balance                
        
        # SUBSTRACT 1 FOR TIMESTEP
        self.episode_len -= 1
        done = self.episode_len <= 0
        
        # CALCULATE IRR, NPV AND CARBON INTENSITY
        total_cash_flow = self.calculate_irr_and_npv(self.pv_costs, exported_revenue, minimised_revenue, current_penalty)
        irr = npf.irr(total_cash_flow) * 100
        npv = npf.npv(0.04 ,total_cash_flow)
            
        # RETURNS AND CALCULATE REWARD
        if self.episode_len == 0:
            irr_fin = irr
            npv_fin = npv

        reward = self.fin_balance_tot / 1000 if done else 0
        
        # FAILURE
         
        survival = self.failure(actions_step)
        self.broke = np.zeros(self.number_of_panels, dtype=np.float32)

        for c, p in enumerate(survival):
            
            if c < self.number_of_panels:

                if p - 1 <= abs(self.episode_len - 24):
                    self.broke[c] = 1

                    self.observation[c] = 0
        
        # DEGRADATION RATE
        # Applying degradation only to panels that are operational (above 0.1 efficiency)
        active_panels = self.observation[:self.number_of_panels] > 0.1
        degradations = np.random.normal(self.deg_mu, self.deg_std, size=self.number_of_panels) / 100
        self.observation[:self.number_of_panels][active_panels] -= degradations[active_panels]
        
        # BATERY DEGRADATION
        self.battery_state = determine_battery_degradation(action_battery, no_cycles)
        
        self.observation[31] = self.battery_state


        
        if not done: 
        
            self.next_cost_per_Wp = self.random_cost_per_Wp[abs(self.episode_len - 25)]
            self.next_import_price = self.random_import_price[abs(self.episode_len - 25)]
            self.next_efficency_develop = self.random_efficency_develop[abs(self.episode_len - 25)]
            next_FIT = self.calculate_FiT(self.episode_len, self.next_import_price)
        
            price_observation = (self.next_import_price - 0.00022499) / (0.0020798 - 0.00022499)
            self.observation[self.number_of_panels] = price_observation

            FiT_observation = (next_FIT - 0.00022499 * 0.33) / (0.0020798 - 0.00022499 * 0.33)
            self.observation[self.number_of_panels + 1] = FiT_observation

            eff_observation = (self.next_efficency_develop - 0.999) / (1.156 - 0.999)
            self.observation[self.number_of_panels + 2] = eff_observation

            cost_per_Wp_observation = (self.next_cost_per_Wp - 0.1712159) / (0.8214356 - 0.1712159) 
            self.observation[self.number_of_panels + 3] = cost_per_Wp_observation
            
            step_battery_CAPEX_obs = (self.step_battery_CAPEX - 0.03) / (4 - 0.03)
            self.observation[32] = step_battery_CAPEX_obs
        
        info = {"step financial balance (eur):": self.fin_balance,
               "total financial balance: (eur)": self.fin_balance_tot,
               "internal rate of return": irr_fin,
               "current_interest": current_penalty,
                "net present value": npv_fin}
         
        
        self.previous_observation = self.observation.copy()
        
        return self.observation, reward, done, False, info

In [72]:
env = TrainEnvironment(AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_price_train_arr, Eff_train_arr, CAPEX_JA_train_arr, Battery_CAPEX_train_arr, battery_degradation_train_arr)
env_test = TestEnvironment(AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_price_test_arr, Eff_test_arr, CAPEX_JA_test_arr, Battery_CAPEX_test_arr, battery_degradation_test_arr)

In [228]:
#check_env(env)

In [229]:
def test3(episodes, environment):    
    for episode in range(episodes):
        done = False
        obs = environment.reset()
        step = 0
        print(obs, "\n")
        while not done:
            step += 1
            random_action = environment.action_space.sample()
            obs, reward, done, trun, info = environment.step(random_action)
            
            
            # Extracting the 2nd and 3rd key-value pairs
            keys = list(info.keys())
            values = list(info.values())

            # Getting the 2nd key-value pair
            zeroth_key = keys[0]
            zeroth_value = values[0]

            # Getting the 3rd key-value pair

            sixth_key = keys[3]
            sixth_value = values[3]
            
            print("STEP:", step)
            print("ACT","\n",  random_action)
            print("OBS","\n",  obs)
            print(zeroth_key, zeroth_value, sixth_key, sixth_value)
            print("\n")

In [230]:
#test3(1, env)

In [231]:
#test1(1000, env)

In [232]:
def make_env(rank: int, seed: int = 0) -> Callable:
    def _init() -> gym.Env:
        random.seed(seed + rank)
        np.random.seed(seed + rank) 
        env = TrainEnvironment(AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_price_train_arr, Eff_train_arr, CAPEX_JA_train_arr, Battery_CAPEX_train_arr)
        env.reset(seed=seed + rank)
        return env

    return _init
# Number of environments to run in parallel
num_cpu = 16
env = SubprocVecEnv([make_env(i) for i in range(num_cpu)])   

In [233]:
import math
def logarithmic_schedule(initial_value, final_value=0.00001):
    """
    Returns a function that computes a logarithmically decreasing value from initial_value to final_value.
    """
    def func(progress_remaining):
        # Avoid taking log of zero by setting a lower limit close to zero
        epsilon = 0.0001
        progress = max(epsilon, 1 - progress_remaining)
        # Calculate the decay factor using a logarithmic scale
        return final_value + (initial_value - final_value) * math.log(1/progress)
    return func


learning_rate = logarithmic_schedule(0.0001)

In [234]:
def linear_schedule(initial_value, final_value=0.00001):
    """
    Returns a function that computes a linearly decreasing value from initial_value to final_value.
    """
    def func(progress_remaining):
        # Calculate the decrease based on the remaining progress
        return final_value + (initial_value - final_value) * progress_remaining
    return func

# Define the learning rate using the linear schedule
learning_rate = linear_schedule(0.0003)

In [235]:
log_path = "./logs/"
eval_callback = EvalCallback(env_test, best_model_save_path = "C:/Users/kubaw/Desktop/DELFT/THESIS/CODE/TEST_MODELS/baterytrina24_high/",
                             log_path = log_path, n_eval_episodes = 750, eval_freq=10000,
                             deterministic=True, render=False)


policy_kwargs = dict(net_arch=dict(pi=[1024, 1024], vf=[1024, 1024]))

In [236]:
model = PPO("MlpPolicy", env, learning_rate = learning_rate, batch_size = 1024, n_epochs = 24,  policy_kwargs = policy_kwargs, gamma = 0.99,  verbose=1, tensorboard_log = "C:/Users/kubaw/Desktop/DELFT/THESIS\CODE/TEST_MODELS/LOGS/logs")
TIMESTEPS = 5000000
model.learn(total_timesteps = TIMESTEPS, callback=eval_callback)

Using cpu device
Logging to C:/Users/kubaw/Desktop/DELFT/THESIS\CODE/TEST_MODELS/LOGS/logs\PPO_395


C:\Users\kubaw\miniforge3\envs\pytorch-env\lib\site-packages\stable_baselines3\common\callbacks.py:414: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.subproc_vec_env.SubprocVecEnv object at 0x000002A096B3F460> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x000002A0A8C91ED0>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


------------------------------
| time/              |       |
|    fps             | 3612  |
|    iterations      | 1     |
|    time_elapsed    | 9     |
|    total_timesteps | 32768 |
------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1048        |
|    iterations           | 2           |
|    time_elapsed         | 62          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.018437067 |
|    clip_fraction        | 0.282       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.89       |
|    explained_variance   | 0.0011      |
|    learning_rate        | 0.000298    |
|    loss                 | 58.6        |
|    n_updates            | 24          |
|    policy_gradient_loss | -0.0313     |
|    value_loss           | 236         |
-----------------------------------------
---------------------------

C:\Users\kubaw\miniforge3\envs\pytorch-env\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=160000, episode_reward=17.71 +/- 8.93
Episode length: 25.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 25          |
|    mean_reward          | 17.7        |
| time/                   |             |
|    total_timesteps      | 160000      |
| train/                  |             |
|    approx_kl            | 0.020656658 |
|    clip_fraction        | 0.319       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.65       |
|    explained_variance   | 0.699       |
|    learning_rate        | 0.000292    |
|    loss                 | 45.5        |
|    n_updates            | 96          |
|    policy_gradient_loss | -0.0375     |
|    value_loss           | 95.9        |
-----------------------------------------
New best mean reward!
-------------------------------
| time/              |        |
|    fps             | 547    |
|    iterations      | 5      |
|    t

...

------------------------------------------
| time/                   |              |
|    fps                  | 599          |
|    iterations           | 105          |
|    time_elapsed         | 5739         |
|    total_timesteps      | 3440640      |
| train/                  |              |
|    approx_kl            | 0.0058969893 |
|    clip_fraction        | 0.0579       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.524       |
|    explained_variance   | 0.613        |
|    learning_rate        | 0.000102     |
|    loss                 | 9.56         |
|    n_updates            | 2496         |
|    policy_gradient_loss | -0.00522     |
|    value_loss           | 18.9         |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 600          |
|    iterations           | 106          |
|    time_elapsed         | 5782         |
|    total_

-----------------------------------------
| time/                   |             |
|    fps                  | 605         |
|    iterations           | 115         |
|    time_elapsed         | 6221        |
|    total_timesteps      | 3768320     |
| train/                  |             |
|    approx_kl            | 0.005667475 |
|    clip_fraction        | 0.0555      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.496      |
|    explained_variance   | 0.621       |
|    learning_rate        | 8.33e-05    |
|    loss                 | 9.01        |
|    n_updates            | 2736        |
|    policy_gradient_loss | -0.00465    |
|    value_loss           | 18.9        |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 606          |
|    iterations           | 116          |
|    time_elapsed         | 6265         |
|    total_timesteps      | 3

------------------------------------------
| time/                   |              |
|    fps                  | 611          |
|    iterations           | 125          |
|    time_elapsed         | 6701         |
|    total_timesteps      | 4096000      |
| train/                  |              |
|    approx_kl            | 0.0042435443 |
|    clip_fraction        | 0.0442       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.46        |
|    explained_variance   | 0.616        |
|    learning_rate        | 6.43e-05     |
|    loss                 | 8.89         |
|    n_updates            | 2976         |
|    policy_gradient_loss | -0.00507     |
|    value_loss           | 18.4         |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 612          |
|    iterations           | 126          |
|    time_elapsed         | 6745         |
|    total_

------------------------------------------
| time/                   |              |
|    fps                  | 615          |
|    iterations           | 135          |
|    time_elapsed         | 7181         |
|    total_timesteps      | 4423680      |
| train/                  |              |
|    approx_kl            | 0.0037662466 |
|    clip_fraction        | 0.0366       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.459       |
|    explained_variance   | 0.607        |
|    learning_rate        | 4.53e-05     |
|    loss                 | 10.8         |
|    n_updates            | 3216         |
|    policy_gradient_loss | -0.00486     |
|    value_loss           | 21.5         |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 616          |
|    iterations           | 136          |
|    time_elapsed         | 7225         |
|    total_

------------------------------------------
| time/                   |              |
|    fps                  | 620          |
|    iterations           | 145          |
|    time_elapsed         | 7661         |
|    total_timesteps      | 4751360      |
| train/                  |              |
|    approx_kl            | 0.0030283923 |
|    clip_fraction        | 0.0261       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.427       |
|    explained_variance   | 0.617        |
|    learning_rate        | 2.63e-05     |
|    loss                 | 8.29         |
|    n_updates            | 3456         |
|    policy_gradient_loss | -0.00404     |
|    value_loss           | 18.8         |
------------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 620        |
|    iterations           | 146        |
|    time_elapsed         | 7704       |
|    total_timesteps 

In [237]:
model.save(r"C:\Users\kubaw\Desktop\DELFT\THESIS\CODE\TEST_MODELS\baterytrina24_high_")

In [238]:
#model = PPO.load(r"C:\Users\kubaw\Desktop\DELFT\THESIS\CODE\TEST_MODELS\bateryJA24\best_model.zip")

In [239]:
evaluate1(1, env_test, model)

Act: [12  0] 
 Obs: [0.9873385  0.8577553  0.92364246 0.8684109  0.9917868  0.93270177
 0.8611209  0.9986229  0.98365194 0.9821843  0.8793439  0.900826
 0.9812651  0.9872371  0.9269218  0.98619074 0.99129    0.9015927
 0.8918074  0.9977231  0.98882157 0.8711458  0.9067533  0.9924551
 0.09120543 0.08876099 0.00876541 0.26855278 0.21556297 0.06014763
 0.3688     0.         0.24433249] 
 Balance 309.72761288268293
Act: [0 0] 
 Obs: [0.9733854  0.85080546 0.91454    0.8582409  0.97990876 0.91184354
 0.85832554 0.99697375 0.97094905 0.97110844 0.86932915 0.9020662
 0.983444   0.9723692  0.91131055 0.97942173 0.96984    0.8942644
 0.8753438  0.98821723 0.97617286 0.867638   0.8948084  0.9788829
 0.09497126 0.09098997 0.00943227 0.2587641  0.04660992 0.06375648
 0.0608     0.         0.24354868] 
 Balance 988.059467291343
Act: [0 0] 
 Obs: [0.96761733 0.82957345 0.89367026 0.854353   0.9706752  0.89915997
 0.8495043  0.99555904 0.9547409  0.9449344  0.8513423  0.89121217
 0.96666974 0.9624315

In [240]:
def evaluate2(episodes, environment, model):
    
    mean_irr = 0
    mean_fin_balance = 0
    irr = 0
    fin_balance = 0
    count = 0
    irr_count = 0
    npv = 0

    for ep in range(episodes):

        obs, _ = environment.reset()  # Unpack the tuple and ignore the info part
        done = False

        while not done:
            action, _ = model.predict(obs)  # Now obs is just the observation array
            obs, reward, done, truncated, info = environment.step(action)
            # Extracting the 2nd and 3rd key-value pairs
            keys = list(info.keys())
            values = list(info.values())

            # Getting the 2nd key-value pair
            second_value = values[1]
            # Getting the 3rd key-value pair
    
            fourth_value = values[4]
        
        fin_balance += second_value
        npv += fourth_value
        count += 1

    mean_fin_balance = fin_balance/count
    mean_npv = npv/count
    
    print(mean_npv, "\n", mean_irr, "\n" )

    environment.close()

In [241]:
evaluate2(10000, env_test, model)

10448.729321015973 
 0 



In [242]:
def basepolicy1(episodes, environment, number_of_panels):
    
    mean_irr = 0
    mean_fin_balance = 0
    irr = 0
    fin_balance = 0
    count = 0
    irr_count = 0
    npv = 0

    for ep in range(episodes):

        obs, _ = environment.reset()  # Unpack the tuple and ignore the info part
        done = False
        
        action = [0, 0]
    

        while not done:
            action[0] = 0
            action[1] = 0
            for i, n in enumerate(obs):
                if i < number_of_panels:
                    if n < 0.7:
                        action[0] += 1
            
            if obs[[31]] == 0:
                action[1] = 1
                
                
            obs, reward, done, truncated, info = environment.step(action)

            # Extracting the 2nd and 3rd key-value pairs
            keys = list(info.keys())
            values = list(info.values())

            # Getting the 2nd key-value pair
            second_value = values[1]

            # Getting the 3rd key-value pair
    
            third_value = values[2]
            fourth_value = values[4]
        
        fin_balance += second_value
        npv += fourth_value
        count += 1
            
    mean_fin_balance = fin_balance/count
    mean_npv = npv/count

    print(mean_npv, "\n", mean_irr, "\n" )

    environment.close()

In [243]:
basepolicy1(10000, env_test, 24)

2712.953634356207 
 0 



In [244]:
# import and modify data

# Assuming the file is a CSV and specifying the correct path and filename
file_path = r"C:\Users\kubaw\Desktop\DELFT\THESIS\CH5"

# Use pandas to read the CSV file
AC_OUTPUT = pd.read_csv(file_path + "/AC_OUTPUT_SunPower")
elec_df = pd.read_csv(file_path + "/hourly_consumption_gemany.csv")
import_price = pd.read_csv(file_path + "/electricity_tariff.csv")

#elec_df = elec_df * 1000
elec_df = elec_df.drop('HourOfYear', axis=1)

elec_df['hour_of_day'] = np.arange(8760) % 24
elec_df['day_of_week'] = np.arange(8760) // 24 % 7  # 0 is Monday, 6 is Sunday

# Define rates
peak_rate = 1.45
normal_rate = 1
off_peak_rate = 0.85

# Function to determine rate based on hour and day
def determine_rate(hour, day):
    if day < 5:  # Monday to Friday
        if 16 <= hour < 21:  # 4pm to 9pm
            return peak_rate
        elif 6 <= hour < 10:  # 7am to 9am and 10am to 3pm
            return normal_rate
        else:  # Off-peak times
            return off_peak_rate
    else:  # Weekend
        if 16 <= hour < 21:  # 4pm to 9pm
            return normal_rate
        else:  # Off-peak times
            return off_peak_rate
    
# Apply the function to each row to determine the rate
elec_df['rate'] = elec_df.apply(lambda row: determine_rate(row['hour_of_day'], row['day_of_week']), axis=1)

import_price_df = import_price.drop(columns=['x'])
import_price_df = import_price_df[:-26]

train_cols = random.sample(list(import_price_df.columns), 7000)
import_price_train = import_price_df[train_cols]
test_cols = [col for col in import_price_df.columns if col not in train_cols]
import_price_test = import_price_df[test_cols]

Eff = pd.read_csv(file_path + "/Efficency_impr")
Eff = (Eff)/100 + 1

CAPEX = pd.read_csv(file_path + "/CAPEX_MAX.csv")
CAPEX_JA = (CAPEX[:26])

train_cols_CAPEX = random.sample(list(CAPEX_JA.columns), 7000)
test_cols_CAPEX = [col for col in CAPEX_JA.columns if col not in train_cols_CAPEX]

CAPEX_JA_train = CAPEX_JA[train_cols_CAPEX]
CAPEX_JA_test = CAPEX_JA[test_cols_CAPEX]

train_cols_Eff = random.sample(list(Eff.columns), 7000)
test_cols_Eff = [col for col in Eff.columns if col not in train_cols_Eff]

Eff_train = Eff[train_cols_Eff]
Eff_test = Eff[test_cols_Eff]

AC_OUTPUT_arr = (np.array(AC_OUTPUT.T)).flatten()

Eff_train_arr = np.array(Eff_train.T)
Eff_test_arr = np.array(Eff_test.T)

CAPEX_JA_train_arr = np.array(CAPEX_JA_train.T)
CAPEX_JA_test_arr = np.array(CAPEX_JA_test.T)

elec_consum_arr = np.array(elec_df["Consumption"])
import_price_rate = np.array(elec_df["rate"])

import_price_train_arr = np.array(import_price_train.T)
import_price_test_arr = np.array(import_price_train.T)

file_path2 = r"C:\Users\kubaw\Desktop\DELFT\THESIS\CH6"

Battery_CAPEX = pd.read_csv(file_path2 + "/batery_CAPES.csv")
Battery_CAPEX = (Battery_CAPEX[:26])


train_cols_Battery_CAPEX = random.sample(list(Battery_CAPEX.columns), 7000)
test_cols_Battery_CAPEX = [col for col in Battery_CAPEX.columns if col not in train_cols_Battery_CAPEX]

Battery_CAPEX_train = Battery_CAPEX[train_cols_Battery_CAPEX]
Battery_CAPEX_test = Battery_CAPEX[test_cols_Battery_CAPEX]

Battery_CAPEX_train_arr = np.array(Battery_CAPEX_train.T)
Battery_CAPEX_test_arr = np.array(Battery_CAPEX_test.T)

In [245]:
class TrainEnvironment(gym.Env):
    def __init__(self, AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_tariff, efficency, CAPEX, Battery_CAPEX):
        
        # Price per watthour
        self.import_price_df = import_tariff
        self.import_price_at_zero = np.float32(0.00035)
        self.import_price_rate = import_price_rate
        
        # Energy Balance
        self.AC_OUTPUT = AC_OUTPUT_arr
        self.elec_df = elec_consum_arr
        self.max_export = 4000
        self.number_of_panels = 24
        
        # Degradation
        self.deg_mu = 1.055
        self.deg_std = 0.555
        
        # Efficency Development
        self.efficency_develop_df = efficency
        self.efficency_at_zero = 1.0
        
        # Costs
        self.power_at_zero = 415
        self.cost_per_Wp_df_at_zero = 0.69
        self.cost_per_Wp_df = CAPEX
        self.initial_other_costs = 150
        
        self.operational_cost = 16.8
        #self.budget_constraint = 1500
        
        self.loan_interest_rate = 1.06
        self.normal_interest_rate = 1.02
        
        self.battery_CAPEX = Battery_CAPEX
        self.battery_CAPEX_at_zero = 1
        
        self.possible_batery_sizes = 1
                        
        # Spaces and length
        self.action_space = spaces.MultiDiscrete([self.number_of_panels + 1, 2])
        
        self.observation_space = spaces.Box(0, 1.25, shape=(self.number_of_panels + 7 + 2,))
        self.episode_len = 25
        self.months_per_timestep = 12
        
    def _get_obs(self):
        
        return self.observation
    
    def calculate_import_export(self, AC_OUTPUT, elec_df, export_price, import_price):
        
        """
        Calculate the annual Wh of energy exported to the grid (exported) and saved (minimised)
        """
        
        batery_state = 1
        minimised_mult = 1
        export_mult = 1
        if self.battery_state > 0 and self.battery_state <=1:
            minimised_mult = 2.155
            export_mult = 0.52
            batery_state = self.battery_state
        
        AC_OUTPUT_tot = self._get_obs()[0:self.number_of_panels].sum() * self.AC_OUTPUT 

        exported = (AC_OUTPUT_tot - self.elec_df).clip(min=0, max = self.max_export)        
        export_revenue = (export_price * exported).sum() * export_mult

        
        minimised = AC_OUTPUT_tot - exported 
        minimised_revenue = (minimised * (self.import_price_rate * import_price)).sum() * minimised_mult * batery_state
        

        return export_revenue, AC_OUTPUT_tot, minimised_revenue    
    
    def reset(self, seed=None):
        
        """
        Reset the environment to the original state at t=1
        """
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)
        
        # Panels
        self.init_obs = np.random.uniform(0, 1, size=self.number_of_panels).astype(np.float32)
        self.init_obs = np.where(self.init_obs < 0.5, 0.0, np.random.uniform(0.85, 1.0, size=self.number_of_panels))

        # Combine all initialization into a single step for efficiency
        self.import_price_at_zero_norm = (self.import_price_at_zero - 0.00022499) / (0.0020798 - 0.00022499)
        self.FiT_at_zero_norm = (self.import_price_at_zero - 0.00022499 * 0.33) / (0.0020798 - 0.00022499 * 0.33)
        self.efficency_at_zero_norm = (self.efficency_at_zero - 0.999) / (1.156 - 0.999)
        self.panel_cost_and_inverter_at_zero_norm = (self.cost_per_Wp_df_at_zero - 0.1712159) / (0.8214356 - 0.1712159)
        
        self.current_budget_constraint = np.random.randint(0, 750)
        self.next_step_budget_constraint = 0
        
        self.battery_CAPEX_at_zero_norm = (self.battery_CAPEX_at_zero - 0.03) / (4 - 0.03)
        
        # Complete observation initialization in one go
        self.observation = np.concatenate([
            self.init_obs,
            [self.import_price_at_zero_norm, self.FiT_at_zero_norm, self.efficency_at_zero_norm, 
             self.panel_cost_and_inverter_at_zero_norm, 0., 0., 0., 0., self.battery_CAPEX_at_zero_norm]
        ]).astype(np.float32)

        self.previous_observation = self.observation.copy()

        # RANDOM IMPORT PRICE
        self.random_import_price = self.import_price_df[np.random.choice(self.import_price_df.shape[0])] 

        # RANDOM EFFICENCY
        self.random_efficency_develop = self.efficency_develop_df[np.random.choice(self.efficency_develop_df.shape[0])]   
        
        # RANDOM COST PER WP
        self.random_cost_per_Wp = self.cost_per_Wp_df[np.random.choice(self.cost_per_Wp_df.shape[0])]   
        
        self.random_battery_CAPEX = self.battery_CAPEX[np.random.choice(self.battery_CAPEX.shape[0])]
        
        self.episode_len = 25  
    
        info = {}
        
        # RESET BALANCES
        self.fin_balance_tot = 0
        self.reward_tot = 0
        self.env_balance_tot = 0
        self.produced = 0
        self.other_costs = 0
        self.FiT = 0.0004
        self.next_FiT = 0.0004

        self.total_cash_flow = []
        self.annual_cash_flow = 0
                
        self.due_loans = [0, 0, 0, 0] 
        self.current_interest = 0
        self.step_total_interest = 1
        self.survival = np.zeros(self.number_of_panels, dtype=np.float32)
        self.resale_values = array_of_zeros = np.zeros(self.number_of_panels, dtype=np.float32)
        
        self.broke = np.zeros(self.number_of_panels, dtype=np.float32)
        
        self.two_year_ago_interest = 0
        self.first_year_interest = []
        self.second_year_interest = [0]
        self.third_year_interest = [0, 0]
        self.fourth_year_interest = [0, 0, 0]
        self.next_year_total = 0
        
        self.battery_state = 0
        
        self.survival = np.zeros(self.number_of_panels, dtype=np.float32)
    
        return self.observation, info
    
    def calculate_resale(self, initial_panel_cost, indices):
        
        self.resale_values[indices] = initial_panel_cost
        
        self.resale_values = self.resale_values * 0.85
        
        for count, i in enumerate(self.broke):
            if i == 1:
                self.resale_values[count] = 0
        
        resale_step = self.resale_values[indices].sum()
        
        return resale_step
    
    def calculate_panel_inv_cost(self, cost_per_Wp):
        
        PW_ep = self.efficency_develop * self.power_at_zero
        
        panel_cost_and_inverter = PW_ep * cost_per_Wp
        
        return panel_cost_and_inverter
        
    def calculate_penalty(self, current_step, annual_expense):
              
        year = 25 - current_step
        
        if year > 0:
            self.current_budget_constraint = self.next_step_budget_constraint    
            
        
        self.current_interest = self.next_year_total
        annual_expense = (-annual_expense)
        value = 0 
        loan = 0
        annual_interest = 0

        if annual_expense > self.current_budget_constraint:
            loan = (self.current_budget_constraint - annual_expense)
            value = annual_expense / self.current_budget_constraint
            periods = 2 if value < 2 else 3 if value < 3 else 4

            annual_interest = loan / periods
            interest_multiplier = 1

            for i in range(4):
                if i < periods:
                    self.due_loans[i] = annual_interest * interest_multiplier
                    interest_multiplier *= self.loan_interest_rate
                else:
                    self.due_loans[i] = 0
        else:
             self.due_loans = [0, 0, 0, 0]
    
        self.first_year_interest.append(self.due_loans[0])
        self.second_year_interest.append(self.due_loans[1])
        self.third_year_interest.append(self.due_loans[2])
        self.fourth_year_interest.append(self.due_loans[3])
    
    
        self.next_year_total = self.first_year_interest[year] + self.second_year_interest[year] + self.third_year_interest[year] + self.fourth_year_interest[year]
        
        self.next_step_budget_constraint = np.random.randint(0, 750) * self.step_total_interest
        current_budget_observation = (self.next_step_budget_constraint - 0 * self.step_total_interest) / (750 * self.step_total_interest - 0 * self.step_total_interest) 
        self.observation[self.number_of_panels + 6] = current_budget_observation
                
        return self.current_interest, self.due_loans, self.next_year_total
        
    def calculate_total_CAPEX(self, action_step, panel_cost_and_inverter):
        """
        Calculate CAPEX each step in a vectorized manner.
        """
        BOS = panel_cost_and_inverter * 0.55
        number_installed = int(np.sum(action_step))

        # Calculate costs from module and inverter
        panel_cost_and_inverter_step = panel_cost_and_inverter * number_installed

        # Calculate other installation costs
        if number_installed == 0:
            other_costs = 0
        elif number_installed == 1:
            other_costs = self.initial_other_costs * self.step_total_interest
        else:
            discounts = 0.9 ** np.arange(number_installed)
            other_costs = (self.initial_other_costs * self.step_total_interest * discounts).sum()

        # Calculate BOS costs using vector operations
        is_new_installation = (self.previous_observation[:number_installed] == 0) & (action_step[:number_installed] == 1)
        is_replacement = (self.previous_observation[:number_installed] > 0) & (action_step[:number_installed] == 1)
        BOS_cost = np.sum(BOS * is_new_installation) + np.sum((BOS / 2) * is_replacement)

        # Sum total CAPEX
        total_CAPEX = panel_cost_and_inverter_step + BOS_cost + other_costs

        return total_CAPEX, panel_cost_and_inverter
        
    def failure(self, actions):
        
        beta = 3  # Shape parameter
        phi = 50  # Scale parameter  

        # Determine which panels are active based on the actions and previous observations.
        if self.episode_len == 24:
            active_panels = (self.observation[:self.number_of_panels] > 0.85)
        else:
            active_panels = (self.observation[:self.number_of_panels] == self.efficency_develop)

        # Calculate lifespan for all active panels at once
        lifespans = np.random.weibull(beta, self.number_of_panels) * phi
        lifespans = np.where(active_panels, lifespans, 0)  # Apply lifespan only to active panels

        # Adjust survival times based on episode length
        self.survival[:self.number_of_panels] = np.where(
            active_panels,
            np.abs(lifespans.astype(int)) + np.abs(self.episode_len - 25),
            self.survival[:self.number_of_panels]
        )

        return self.survival

    def calculate_FiT(self, episodes, import_price):
            
        self.FiT = import_price
            
        if episodes == 25:
            self.FiT = self.FiT
            
        elif episodes == 24 or episodes == 23:
            self.FiT = self.FiT * 0.64
            
        elif episodes == 22:
            self.FiT = self.FiT * 0.46
            
        elif episodes == 21:
            self.FiT = self.FiT * 0.55
            
        elif episodes < 20:
            self.FiT = self.FiT * 0.33
            
        elif episodes == 20:
            self.FiT = self.FiT * 0.37
            
        return self.FiT
                        
    def calculate_battery_CAPEX(self, action_battery):
        
        five_kw_PRICE = 3000

        installation = 1000 * self.step_total_interest
        
        this_battery_CAPEX = five_kw_PRICE * self.step_battery_CAPEX
        
        battery_bos = this_battery_CAPEX * 0.5
        
        if self.previous_observation[19] == 0:
            battery_bos = this_battery_CAPEX * 0.5
        else:
            battery_bos = 0
        
        total_batery_cost = (this_battery_CAPEX +  battery_bos + installation) * action_battery
        
        return total_batery_cost
        
    def step(self, action):
        
        """
        defines actions, reward etc.
        """
        
        # RESET THE ANNUAL BALANCES
        self.total_CAPEX = 0
        self.pv_costs = 0
        self.fin_balance = 0
        self.number_installed = 0
        current_penalty = 0
        self.other_costs = 0
        battery_costs = 0
        next_step_penalty = 0
        self.step_total_interest = self.step_total_interest * self.normal_interest_rate
        current_operational_costs = self.operational_cost * self.step_total_interest
        
        #***
        self.step_battery_CAPEX = self.random_battery_CAPEX[abs(self.episode_len - 25)]
        
        self.cost_per_Wp = self.random_cost_per_Wp[abs(self.episode_len - 25)]
        self.import_price = self.random_import_price[abs(self.episode_len - 25)]
        self.efficency_develop = self.random_efficency_develop[abs(self.episode_len - 25)]
           
        self.panel_cost_and_inverter = self.calculate_panel_inv_cost(self.cost_per_Wp)
        FiT = self.calculate_FiT(self.episode_len, self.import_price)
        
        reward = 0   
        actions_step = np.random.rand(12)
        
        #  ***
        action_battery = action[1]
        action = action[0]
        
        # Find indices of the lowest 'action' values in previous_observation
        indices = np.argsort(self.previous_observation[:self.number_of_panels])[:action]

        # Replace these indices in the observation with efficiency_develop
        self.observation[:self.number_of_panels][indices] = self.efficency_develop

        # Copy over the other values from previous_observation to observation
        mask = np.ones(len(self.previous_observation[:self.number_of_panels]), dtype=bool)
        mask[indices] = False
        self.observation[:self.number_of_panels][mask] = self.previous_observation[:self.number_of_panels][mask]

        replaced_panels = np.zeros(len(self.previous_observation[:self.number_of_panels]), dtype=int)
        replaced_panels[indices] = 1

        instaltion = (self.observation[:self.number_of_panels] > 0).astype(int)
        self.pv_costs -= instaltion.sum() * current_operational_costs

        actions_step = np.array(replaced_panels)
        
        if action_battery == 1:           
            self.observation[31] = 1.
            self.battery_state = 1
            
        if action > 0:
            step_CAPEX, panel_cost_and_inverter = self.calculate_total_CAPEX(actions_step, self.panel_cost_and_inverter)
            self.pv_costs -= step_CAPEX
            
        else:
            panel_cost_and_inverter = 0
            
        battery_costs = self.calculate_battery_CAPEX(action_battery)
        
        self.pv_costs -= battery_costs

        
        next_observation = self._get_obs()

        # Calculate the Reslae value
        resale = self.calculate_resale(panel_cost_and_inverter, indices)
        
        self.pv_costs += resale
 
        
        # CALCULATE THE BUDGET INTEREST
        current_penalty, due_loans, next_step_penalty = self.calculate_penalty(self.episode_len, self.pv_costs)

        
        # CALCULATE THE ENERGY YIELD
        exported_revenue, AC_OUTPUT_tot, minimised_revenue = self.calculate_import_export(self.AC_OUTPUT, 
                                                                          self.elec_df, FiT, self.import_price)        
        
        pv_costs_observation = - self.pv_costs / 9000
        self.observation[self.number_of_panels + 4] = pv_costs_observation
        
        next_step_penalty_observation = - next_step_penalty / 7000
        self.observation[self.number_of_panels + 5] = next_step_penalty_observation
        
        
        # CALCULATE STEP BALANCES
        self.fin_balance += self.pv_costs
        self.fin_balance += current_penalty
        self.fin_balance += float(exported_revenue + minimised_revenue)
        
        # CALCULATE TOTAL BALANCES
        self.fin_balance_tot += self.fin_balance                
        
        # SUBSTRACT 1 FOR TIMESTEP
        self.episode_len -= 1
        done = self.episode_len <= 0
        
        #reward = self.fin_balance_tot / 1000 if done else 0
        reward = self.fin_balance / 1000
        
        # FAILURE
        self.broke = np.zeros(self.number_of_panels, dtype=np.float32)
        survival = self.failure(actions_step)
        
        for c, p in enumerate(survival):
            
            if c < self.number_of_panels:

                if p - 1 <= abs(self.episode_len - 24):
                    self.broke[c] = 1
                    self.observation[c] = 0
        
        # DEGRADATION RATE
        # Applying degradation only to panels that are operational (above 0.1 efficiency)
        active_panels = self.observation[:self.number_of_panels] > 0.1
        degradations = np.random.normal(self.deg_mu, self.deg_std, size=self.number_of_panels) / 100
        self.observation[:self.number_of_panels][active_panels] -= degradations[active_panels]
        
        # BATERY DEGRADATION
        if self.observation[31] > 0:
            self.battery_state -= 0.0145
            if self.battery_state < 0.8:
                self.battery_state = 0
        
        self.observation[31] = self.battery_state
        
        if not done: 
        
            self.next_cost_per_Wp = self.random_cost_per_Wp[abs(self.episode_len - 25)]
            self.next_import_price = self.random_import_price[abs(self.episode_len - 25)]
            self.next_efficency_develop = self.random_efficency_develop[abs(self.episode_len - 25)]
            next_FIT = self.calculate_FiT(self.episode_len, self.next_import_price)
        
            price_observation = (self.next_import_price - 0.00022499) / (0.0020798 - 0.00022499)
            self.observation[self.number_of_panels] = price_observation

            FiT_observation = (next_FIT - 0.00022499 * 0.33) / (0.0020798 - 0.00022499 * 0.33)
            self.observation[self.number_of_panels + 1] = FiT_observation

            eff_observation = (self.next_efficency_develop - 0.999) / (1.156 - 0.999)
            self.observation[self.number_of_panels + 2] = eff_observation

            cost_per_Wp_observation = (self.next_cost_per_Wp - 0.1712159) / (0.8214356 - 0.1712159) 
            self.observation[self.number_of_panels + 3] = cost_per_Wp_observation
            
            step_battery_CAPEX_obs = (self.step_battery_CAPEX - 0.03) / (4 - 0.03)
            self.observation[32] = step_battery_CAPEX_obs
        
        info = {"step financial balance (eur):": battery_costs,
               "total financial balance: (eur)": minimised_revenue,
               "internal rate of return": 0,
               "current_interest": resale,
                "net present value": 0}
         
        
        self.previous_observation = self.observation.copy()
        
        return self.observation, reward, done, False, info

In [246]:
class TestEnvironment(gym.Env):
    def __init__(self, AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_tariff, efficency, CAPEX, Battery_CAPEX):
        
        # Price per watthour
        self.import_price_df = import_tariff
        self.import_price_at_zero = np.float32(0.00035)
        self.import_price_rate = import_price_rate
        
        # Energy Balance
        self.AC_OUTPUT = AC_OUTPUT_arr
        self.elec_df = elec_consum_arr
        self.max_export = 4000
        self.number_of_panels = 24
        
        # Degradation
        self.deg_mu = 1.055
        self.deg_std = 0.555
        
        # Efficency Development
        self.efficency_develop_df = efficency
        self.efficency_at_zero = 1.0
        
        # Costs
        self.power_at_zero = 415
        self.cost_per_Wp_df_at_zero = 0.69
        self.cost_per_Wp_df = CAPEX
        self.initial_other_costs = 150
        
        self.operational_cost = 16.8
        #self.budget_constraint = 1500
        
        self.loan_interest_rate = 1.06
        self.normal_interest_rate = 1.02
        
        self.battery_CAPEX = Battery_CAPEX
        self.battery_CAPEX_at_zero = 1
            
        # Spaces and length
        self.action_space = spaces.MultiDiscrete([self.number_of_panels + 1, 2])
        self.observation_space = spaces.Box(0, 1.25, shape=(self.number_of_panels + 7 + 2,))
        self.episode_len = 25
        self.months_per_timestep = 12
        
    def _get_obs(self):
        
        return self.observation
    
    def calculate_import_export(self, AC_OUTPUT, elec_df, export_price, import_price):
        
        """
        Calculate the annual Wh of energy exported to the grid (exported) and saved (minimised)
        """
        
        batery_state = 1
        minimised_mult = 1
        export_mult = 1
        if self.battery_state > 0:
            minimised_mult = 2.175
            export_mult = 0.52
            batery_state = self.battery_state

        
        AC_OUTPUT_tot = self._get_obs()[0:self.number_of_panels].sum() * self.AC_OUTPUT 

        exported = (AC_OUTPUT_tot - self.elec_df).clip(min=0, max = self.max_export)        
        export_revenue = (export_price * exported).sum() * export_mult

        
        minimised = AC_OUTPUT_tot - exported 
        minimised_revenue = (minimised * (self.import_price_rate * import_price)).sum() * minimised_mult * batery_state
        

        return export_revenue, AC_OUTPUT_tot, minimised_revenue
    
    def reset(self, seed=None):
        
        """
        Reset the environment to the original state at t=1
        """
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)
            
        # Panels
        self.init_obs = np.random.uniform(0, 1, size=self.number_of_panels).astype(np.float32)
        self.init_obs = np.where(self.init_obs < 0.5, 0.0, np.random.uniform(0.85, 1.0, size=self.number_of_panels))

        # Combine all initialization into a single step for efficiency
        self.import_price_at_zero_norm = (self.import_price_at_zero - 0.00022499) / (0.0020798 - 0.00022499)
        self.FiT_at_zero_norm = (self.import_price_at_zero - 0.00022499 * 0.33) / (0.0020798 - 0.00022499 * 0.33)
        self.efficency_at_zero_norm = (self.efficency_at_zero - 0.999) / (1.156 - 0.999)
        self.panel_cost_and_inverter_at_zero_norm = (self.cost_per_Wp_df_at_zero - 0.1712159) / (0.8214356 - 0.1712159)
        
        self.current_budget_constraint = np.random.randint(0, 750)
        self.next_step_budget_constraint = 0
        
        
        self.battery_CAPEX_at_zero_norm = (self.battery_CAPEX_at_zero - 0.03) / (4 - 0.03)
        
        # Complete observation initialization in one go
        self.observation = np.concatenate([
            self.init_obs,
            [self.import_price_at_zero_norm, self.FiT_at_zero_norm, self.efficency_at_zero_norm, 
             self.panel_cost_and_inverter_at_zero_norm, 0., 0., 0., 0., self.battery_CAPEX_at_zero_norm]
        ]).astype(np.float32)

        self.previous_observation = self.observation.copy()

        # RANDOM IMPORT PRICE
        self.random_import_price = self.import_price_df[np.random.choice(self.import_price_df.shape[0])] 

        # RANDOM EFFICENCY
        self.random_efficency_develop = self.efficency_develop_df[np.random.choice(self.efficency_develop_df.shape[0])]   
        
        # RANDOM COST PER WP
        self.random_cost_per_Wp = self.cost_per_Wp_df[np.random.choice(self.cost_per_Wp_df.shape[0])]   
        
        self.random_battery_CAPEX = self.battery_CAPEX[np.random.choice(self.battery_CAPEX.shape[0])]
        
        self.episode_len = 25  
    
        info = {}
        
        # RESET BALANCES
        self.fin_balance_tot = 0
        self.reward_tot = 0
        self.env_balance_tot = 0
        self.produced = 0
        self.other_costs = 0
        self.FiT = 0.0004
        self.next_FiT = 0.0004
        self.resale_values = array_of_zeros = np.zeros(self.number_of_panels, dtype=np.float32)
        
        self.broke = np.zeros(self.number_of_panels, dtype=np.float32)
        self.total_cash_flow = []
        self.annual_cash_flow = 0
                
        self.due_loans = [0, 0, 0, 0] 
        self.current_interest = 0
        self.step_total_interest = 1
        
        self.two_year_ago_interest = 0
        self.first_year_interest = []
        self.second_year_interest = [0]
        self.third_year_interest = [0, 0]
        self.fourth_year_interest = [0, 0, 0]
        self.next_year_total = 0
        self.battery_state = 0

        self.survival = np.zeros(self.number_of_panels, dtype=np.float32)
    
        return self.observation, info
    
    def calculate_resale(self, initial_panel_cost, indices):
        
        self.resale_values[indices] = initial_panel_cost
        
        self.resale_values = self.resale_values * 0.85
        
        for count, i in enumerate(self.broke):
            if i == 1:
                self.resale_values[count] = 0
        
        resale_step = self.resale_values[indices].sum()
        
        return resale_step
    
    def calculate_panel_inv_cost(self, cost_per_Wp):
        
        PW_ep = self.efficency_develop * self.power_at_zero
        
        panel_cost_and_inverter = PW_ep * cost_per_Wp
        
        return panel_cost_and_inverter
    
    def calculate_irr_and_npv(self, pv_cost, minimised_revenue, export_revenue, penalty):
                
        """
        Calculates total cash flow of the project needed for the internal rate of return
        """ 
        self.expences = 0
        self.annual_cash_flow = 0
        initial_cost = 0
        
        self.expences = pv_cost
        self.annual_cash_flow = self.expences + export_revenue + minimised_revenue + penalty
        initial_cost_q, x = self.calculate_total_CAPEX(self.init_obs, self.panel_cost_and_inverter)
        initial_cost = - initial_cost_q
        
        if self.episode_len == 24:
            self.total_cash_flow.append(initial_cost + self.annual_cash_flow) 
        else:
            self.total_cash_flow.append(self.annual_cash_flow) 
        
        return self.total_cash_flow
        
    def calculate_penalty(self, current_step, annual_expense):
              
        year = 25 - current_step
        
        if year > 0:
            self.current_budget_constraint = self.next_step_budget_constraint    
            
        
        self.current_interest = self.next_year_total
        annual_expense = (-annual_expense)
        value = 0 
        loan = 0
        annual_interest = 0

        if annual_expense > self.current_budget_constraint:
            loan = (self.current_budget_constraint - annual_expense)
            value = annual_expense / self.current_budget_constraint
            periods = 2 if value < 2 else 3 if value < 3 else 4

            annual_interest = loan / periods
            interest_multiplier = 1

            for i in range(4):
                if i < periods:
                    self.due_loans[i] = annual_interest * interest_multiplier
                    interest_multiplier *= self.loan_interest_rate
                else:
                    self.due_loans[i] = 0
        else:
             self.due_loans = [0, 0, 0, 0]
    
        self.first_year_interest.append(self.due_loans[0])
        self.second_year_interest.append(self.due_loans[1])
        self.third_year_interest.append(self.due_loans[2])
        self.fourth_year_interest.append(self.due_loans[3])
    
    
        self.next_year_total = self.first_year_interest[year] + self.second_year_interest[year] + self.third_year_interest[year] + self.fourth_year_interest[year]
        
        self.next_step_budget_constraint = np.random.randint(0, 750) * self.step_total_interest
        current_budget_observation = (self.next_step_budget_constraint - 0 * self.step_total_interest) / (750 * self.step_total_interest - 0 * self.step_total_interest) 
        self.observation[self.number_of_panels + 6] = current_budget_observation
                
        return self.current_interest, self.due_loans, self.next_year_total
        
    def calculate_total_CAPEX(self, action_step, panel_cost_and_inverter):
        """
        Calculate CAPEX each step in a vectorized manner.
        """
        BOS = panel_cost_and_inverter * 0.55
        number_installed = int(np.sum(action_step))

        # Calculate costs from module and inverter
        panel_cost_and_inverter_step = panel_cost_and_inverter * number_installed

        # Calculate other installation costs
        if number_installed == 0:
            other_costs = 0
        elif number_installed == 1:
            other_costs = self.initial_other_costs * self.step_total_interest
        else:
            discounts = 0.9 ** np.arange(number_installed)
            other_costs = (self.initial_other_costs * self.step_total_interest * discounts).sum()

        # Calculate BOS costs using vector operations
        is_new_installation = (self.previous_observation[:number_installed] == 0) & (action_step[:number_installed] == 1)
        is_replacement = (self.previous_observation[:number_installed] > 0) & (action_step[:number_installed] == 1)
        BOS_cost = np.sum(BOS * is_new_installation) + np.sum((BOS / 2) * is_replacement)

        # Sum total CAPEX
        total_CAPEX = panel_cost_and_inverter_step + BOS_cost + other_costs

        return total_CAPEX, panel_cost_and_inverter
        
    def failure(self, actions):
        
        beta = 3  # Shape parameter
        phi = 50  # Scale parameter  

        # Determine which panels are active based on the actions and previous observations.
        if self.episode_len == 24:
            active_panels = (self.observation[:self.number_of_panels] > 0.85)
        else:
            active_panels = (self.observation[:self.number_of_panels] == self.efficency_develop)

        # Calculate lifespan for all active panels at once
        lifespans = np.random.weibull(beta, self.number_of_panels) * phi
        lifespans = np.where(active_panels, lifespans, 0)  # Apply lifespan only to active panels

        # Adjust survival times based on episode length
        self.survival[:self.number_of_panels] = np.where(
            active_panels,
            np.abs(lifespans.astype(int)) + np.abs(self.episode_len - 25),
            self.survival[:self.number_of_panels]
        )

        return self.survival

    def calculate_FiT(self, episodes, import_price):
            
        self.FiT = import_price
            
        if episodes == 25:
            self.FiT = self.FiT
            
        elif episodes == 24 or episodes == 23:
            self.FiT = self.FiT * 0.64
            
        elif episodes == 22:
            self.FiT = self.FiT * 0.46
            
        elif episodes == 21:
            self.FiT = self.FiT * 0.55
            
        elif episodes < 20:
            self.FiT = self.FiT * 0.33
            
        elif episodes == 20:
            self.FiT = self.FiT * 0.37
            
        return self.FiT
    
    def calculate_battery_CAPEX(self, action_battery):
        
        five_kw_PRICE = 3000

        installation = 1000 * self.step_total_interest
        
        this_battery_CAPEX = five_kw_PRICE * self.step_battery_CAPEX
        
        battery_bos = this_battery_CAPEX * 0.5
        
        if self.previous_observation[19] == 0:
            battery_bos = this_battery_CAPEX * 0.5
        else:
            battery_bos = 0
        
        total_batery_cost = (this_battery_CAPEX +  battery_bos + installation) * action_battery
        
        return total_batery_cost
                  
    def step(self, action):
        
        """
        defines actions, reward etc.
        """
        
        # RESET THE ANNUAL BALANCES
        self.total_CAPEX = 0
        self.pv_costs = 0
        self.fin_balance = 0
        self.number_installed = 0
        irr_fin = 0
        npv_fin = 0
        battery_costs = 0
        current_penalty = 0
        self.other_costs = 0
        next_step_penalty = 0
        self.step_total_interest = self.step_total_interest * self.normal_interest_rate
        current_operational_costs = self.operational_cost * self.step_total_interest
        self.step_battery_CAPEX = self.random_battery_CAPEX[abs(self.episode_len - 25)]

        
        self.cost_per_Wp = self.random_cost_per_Wp[abs(self.episode_len - 25)]
        self.import_price = self.random_import_price[abs(self.episode_len - 25)]
        self.efficency_develop = self.random_efficency_develop[abs(self.episode_len - 25)]
           
        self.panel_cost_and_inverter = self.calculate_panel_inv_cost(self.cost_per_Wp)
        FiT = self.calculate_FiT(self.episode_len, self.import_price)
        
        reward = 0   
        actions_step = np.random.rand(self.number_of_panels + 1)
        
        
        action_battery = action[1]
        action = action[0]
        
        # Find indices of the lowest 'action' values in previous_observation
        indices = np.argsort(self.previous_observation[:self.number_of_panels])[:action]

        # Replace these indices in the observation with efficiency_develop
        self.observation[:self.number_of_panels][indices] = self.efficency_develop

        # Copy over the other values from previous_observation to observation
        mask = np.ones(len(self.previous_observation[:self.number_of_panels]), dtype=bool)
        mask[indices] = False
        self.observation[:self.number_of_panels][mask] = self.previous_observation[:self.number_of_panels][mask]

        replaced_panels = np.zeros(len(self.previous_observation[:self.number_of_panels]), dtype=int)
        replaced_panels[indices] = 1

        instaltion = (self.observation[:self.number_of_panels] > 0).astype(int)
        self.pv_costs -= instaltion.sum() * current_operational_costs

        actions_step = np.array(replaced_panels)
        
        if action_battery == 1:           
            self.observation[31] = 1.
            self.battery_state = 1
            
        if action > 0:
            step_CAPEX, panel_cost_and_inverter = self.calculate_total_CAPEX(actions_step, self.panel_cost_and_inverter)
            self.pv_costs -= step_CAPEX
            
        else:
            panel_cost_and_inverter = 0
            
        battery_costs = self.calculate_battery_CAPEX(action_battery)
        
        self.pv_costs -= battery_costs
        next_observation = self._get_obs()

        # Calculate the Reslae value
        resale = self.calculate_resale(panel_cost_and_inverter, indices) #  ***
        
        self.pv_costs += resale

        
        # CALCULATE THE BUDGET INTEREST
        current_penalty, due_loans, next_step_penalty = self.calculate_penalty(self.episode_len, self.pv_costs)
        
        
        # CALCULATE THE ENERGY YIELD
        exported_revenue, AC_OUTPUT_tot, minimised_revenue = self.calculate_import_export(self.AC_OUTPUT, 
                                                                          self.elec_df, FiT, self.import_price)        
        
        pv_costs_observation = - self.pv_costs / 9000
        self.observation[self.number_of_panels + 4] = pv_costs_observation
        
        next_step_penalty_observation = - next_step_penalty / 7000
        self.observation[self.number_of_panels + 5] = next_step_penalty_observation
        
        
        # CALCULATE STEP BALANCES
        self.fin_balance += self.pv_costs
        self.fin_balance += current_penalty
        self.fin_balance += float(exported_revenue + minimised_revenue)
        
        # CALCULATE TOTAL BALANCES
        self.fin_balance_tot += self.fin_balance                
        
        # SUBSTRACT 1 FOR TIMESTEP
        self.episode_len -= 1
        done = self.episode_len <= 0
        
        # CALCULATE IRR, NPV AND CARBON INTENSITY
        total_cash_flow = self.calculate_irr_and_npv(self.pv_costs, exported_revenue, minimised_revenue, current_penalty)
        irr = npf.irr(total_cash_flow) * 100
        npv = npf.npv(0.04 ,total_cash_flow)
            
        # RETURNS AND CALCULATE REWARD
        if self.episode_len == 0:
            irr_fin = irr
            npv_fin = npv

        reward = self.fin_balance_tot / 1000 if done else 0
        
        # FAILURE
         
        survival = self.failure(actions_step)
        self.broke = np.zeros(self.number_of_panels, dtype=np.float32)

        for c, p in enumerate(survival):
            
            if c < self.number_of_panels:

                if p - 1 <= abs(self.episode_len - 24):
                    self.broke[c] = 1

                    self.observation[c] = 0
        
        # DEGRADATION RATE
        # Applying degradation only to panels that are operational (above 0.1 efficiency)
        active_panels = self.observation[:self.number_of_panels] > 0.1
        degradations = np.random.normal(self.deg_mu, self.deg_std, size=self.number_of_panels) / 100
        self.observation[:self.number_of_panels][active_panels] -= degradations[active_panels]
        
        # BATERY DEGRADATION
        if self.observation[31] > 0:
            self.battery_state -= 0.0145
            if self.battery_state < 0.8:
                self.battery_state = 0
        
        self.observation[31] = self.battery_state

        
        if not done: 
        
            self.next_cost_per_Wp = self.random_cost_per_Wp[abs(self.episode_len - 25)]
            self.next_import_price = self.random_import_price[abs(self.episode_len - 25)]
            self.next_efficency_develop = self.random_efficency_develop[abs(self.episode_len - 25)]
            next_FIT = self.calculate_FiT(self.episode_len, self.next_import_price)
        
            price_observation = (self.next_import_price - 0.00022499) / (0.0020798 - 0.00022499)
            self.observation[self.number_of_panels] = price_observation

            FiT_observation = (next_FIT - 0.00022499 * 0.33) / (0.0020798 - 0.00022499 * 0.33)
            self.observation[self.number_of_panels + 1] = FiT_observation

            eff_observation = (self.next_efficency_develop - 0.999) / (1.156 - 0.999)
            self.observation[self.number_of_panels + 2] = eff_observation

            cost_per_Wp_observation = (self.next_cost_per_Wp - 0.1712159) / (0.8214356 - 0.1712159) 
            self.observation[self.number_of_panels + 3] = cost_per_Wp_observation
            
            step_battery_CAPEX_obs = (self.step_battery_CAPEX - 0.03) / (4 - 0.03)
            self.observation[32] = step_battery_CAPEX_obs
        
        info = {"step financial balance (eur):": self.fin_balance,
               "total financial balance: (eur)": self.fin_balance_tot,
               "internal rate of return": irr_fin,
               "current_interest": current_penalty,
                "net present value": npv_fin}
         
        
        self.previous_observation = self.observation.copy()
        
        return self.observation, reward, done, False, info

In [247]:
env = TrainEnvironment(AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_price_train_arr, Eff_train_arr, CAPEX_JA_train_arr, Battery_CAPEX_train_arr)
env_test = TestEnvironment(AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_price_test_arr, Eff_test_arr, CAPEX_JA_test_arr, Battery_CAPEX_test_arr)

In [248]:
def make_env(rank: int, seed: int = 0) -> Callable:
    def _init() -> gym.Env:
        random.seed(seed + rank)
        np.random.seed(seed + rank) 
        env = TrainEnvironment(AC_OUTPUT_arr, elec_consum_arr, import_price_rate, import_price_train_arr, Eff_train_arr, CAPEX_JA_train_arr, Battery_CAPEX_train_arr)
        env.reset(seed=seed + rank)
        return env

    return _init
# Number of environments to run in parallel
num_cpu = 16
env = SubprocVecEnv([make_env(i) for i in range(num_cpu)])   

In [249]:
log_path = "./logs/"
eval_callback = EvalCallback(env_test, best_model_save_path = "C:/Users/kubaw/Desktop/DELFT/THESIS/CODE/TEST_MODELS/bateryMAX24_low/",
                             log_path = log_path, n_eval_episodes = 750, eval_freq=10000,
                             deterministic=True, render=False)


policy_kwargs = dict(net_arch=dict(pi=[1024, 1024], vf=[1024, 1024]))

In [250]:
model = PPO("MlpPolicy", env, learning_rate = learning_rate, batch_size = 1024, n_epochs = 24,  policy_kwargs = policy_kwargs, gamma = 0.99,  verbose=1, tensorboard_log = "C:/Users/kubaw/Desktop/DELFT/THESIS\CODE/TEST_MODELS/LOGS/logs")
TIMESTEPS = 5000000
model.learn(total_timesteps = TIMESTEPS, callback=eval_callback)

Using cpu device
Logging to C:/Users/kubaw/Desktop/DELFT/THESIS\CODE/TEST_MODELS/LOGS/logs\PPO_396


C:\Users\kubaw\miniforge3\envs\pytorch-env\lib\site-packages\stable_baselines3\common\callbacks.py:414: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.subproc_vec_env.SubprocVecEnv object at 0x000002A0A8BABAF0> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x000002A08C03C7C0>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


------------------------------
| time/              |       |
|    fps             | 3115  |
|    iterations      | 1     |
|    time_elapsed    | 10    |
|    total_timesteps | 32768 |
------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1225        |
|    iterations           | 2           |
|    time_elapsed         | 53          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.014258133 |
|    clip_fraction        | 0.2         |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.9        |
|    explained_variance   | 0.000881    |
|    learning_rate        | 0.000298    |
|    loss                 | 511         |
|    n_updates            | 24          |
|    policy_gradient_loss | -0.0167     |
|    value_loss           | 1.05e+03    |
-----------------------------------------
---------------------------

C:\Users\kubaw\miniforge3\envs\pytorch-env\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
C:\Users\kubaw\AppData\Local\Temp\ipykernel_10116\1914628558.py:213: RuntimeWarning: divide by zero encountered in scalar divide
  value = annual_expense / self.current_budget_constraint


Eval num_timesteps=160000, episode_reward=14.92 +/- 12.57
Episode length: 25.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 25          |
|    mean_reward          | 14.9        |
| time/                   |             |
|    total_timesteps      | 160000      |
| train/                  |             |
|    approx_kl            | 0.014524533 |
|    clip_fraction        | 0.215       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.72       |
|    explained_variance   | -1.67e-06   |
|    learning_rate        | 0.000292    |
|    loss                 | 547         |
|    n_updates            | 96          |
|    policy_gradient_loss | -0.0146     |
|    value_loss           | 1.08e+03    |
-----------------------------------------
New best mean reward!
-------------------------------
| time/              |        |
|    fps             | 800    |
|    iterations      | 5      |
|    

...

------------------------------------------
| time/                   |              |
|    fps                  | 700          |
|    iterations           | 105          |
|    time_elapsed         | 4910         |
|    total_timesteps      | 3440640      |
| train/                  |              |
|    approx_kl            | 0.0026935271 |
|    clip_fraction        | 0.03         |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.72        |
|    explained_variance   | 0.645        |
|    learning_rate        | 0.000102     |
|    loss                 | 51.9         |
|    n_updates            | 2496         |
|    policy_gradient_loss | -0.0014      |
|    value_loss           | 90.6         |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 701          |
|    iterations           | 106          |
|    time_elapsed         | 4952         |
|    total_

------------------------------------------
| time/                   |              |
|    fps                  | 700          |
|    iterations           | 115          |
|    time_elapsed         | 5381         |
|    total_timesteps      | 3768320      |
| train/                  |              |
|    approx_kl            | 0.0031509367 |
|    clip_fraction        | 0.0224       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.696       |
|    explained_variance   | 0.634        |
|    learning_rate        | 8.33e-05     |
|    loss                 | 40           |
|    n_updates            | 2736         |
|    policy_gradient_loss | -0.00131     |
|    value_loss           | 82.1         |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 700          |
|    iterations           | 116          |
|    time_elapsed         | 5423         |
|    total_

------------------------------------------
| time/                   |              |
|    fps                  | 700          |
|    iterations           | 125          |
|    time_elapsed         | 5848         |
|    total_timesteps      | 4096000      |
| train/                  |              |
|    approx_kl            | 0.0022213054 |
|    clip_fraction        | 0.022        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.655       |
|    explained_variance   | 0.645        |
|    learning_rate        | 6.43e-05     |
|    loss                 | 41.9         |
|    n_updates            | 2976         |
|    policy_gradient_loss | -0.00147     |
|    value_loss           | 84.8         |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 700          |
|    iterations           | 126          |
|    time_elapsed         | 5891         |
|    total_

------------------------------------------
| time/                   |              |
|    fps                  | 700          |
|    iterations           | 135          |
|    time_elapsed         | 6317         |
|    total_timesteps      | 4423680      |
| train/                  |              |
|    approx_kl            | 0.0024945964 |
|    clip_fraction        | 0.0234       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.657       |
|    explained_variance   | 0.645        |
|    learning_rate        | 4.53e-05     |
|    loss                 | 46           |
|    n_updates            | 3216         |
|    policy_gradient_loss | -0.00166     |
|    value_loss           | 88.3         |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 700          |
|    iterations           | 136          |
|    time_elapsed         | 6359         |
|    total_

------------------------------------------
| time/                   |              |
|    fps                  | 699          |
|    iterations           | 145          |
|    time_elapsed         | 6791         |
|    total_timesteps      | 4751360      |
| train/                  |              |
|    approx_kl            | 0.0019089599 |
|    clip_fraction        | 0.00996      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.583       |
|    explained_variance   | 0.641        |
|    learning_rate        | 2.63e-05     |
|    loss                 | 44.1         |
|    n_updates            | 3456         |
|    policy_gradient_loss | -0.000878    |
|    value_loss           | 83.6         |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 699          |
|    iterations           | 146          |
|    time_elapsed         | 6835         |
|    total_

In [251]:
model.save(r"C:\Users\kubaw\Desktop\DELFT\THESIS\CODE\TEST_MODELS\bateryMAX24_low")

In [252]:
model = PPO.load(r"C:\Users\kubaw\Desktop\DELFT\THESIS\CODE\TEST_MODELS\bateryMAX24_low.zip")

In [253]:
evaluate2(10000, env_test, model)

C:\Users\kubaw\AppData\Local\Temp\ipykernel_10116\1914628558.py:213: RuntimeWarning: divide by zero encountered in scalar divide
  value = annual_expense / self.current_budget_constraint


15330.806920518813 
 0 



In [254]:
basepolicy1(10000, env_test, 24)

C:\Users\kubaw\AppData\Local\Temp\ipykernel_10116\1914628558.py:213: RuntimeWarning: divide by zero encountered in scalar divide
  value = annual_expense / self.current_budget_constraint


12150.145309023697 
 0 

